
# -----------------------------------------------------------------------------
# Author: [Yongzhang Yu]
# Description: [Fig_6a_co_classification_risk_prediction_model]
# -----------------------------------------------------------------------------

In [ ]:
# env:r43base_rna

In [1]:
rm(list=ls())

In [7]:
.libPaths(c("/share/home/wyz/miniconda3/envs/r43base_rna/lib/R/library","/share/home/bioinfo/anaconda3/envs/R-4.2.3/lib/R/library"))

In [8]:
#引用包
library(survival)
library(caret)
library("glmnet")
library(survminer)
library("survivalROC")

In [7]:
output_dir = "/share/home/wyz/Project/result/2025/202507/20250718_1_Risk_Model_Build/20250828_1_Risk_Model_Build_Union_420_split_clinlic_ROC"

In [5]:
input_f = "/share/home/wyz/Project/result/2025/202507/20250716_1_Risk_Model_Build/20250718_1_Model_input_data/20250718_1_Model_input_data_cell_percent_sample_1307.xls"

## Read Data

In [6]:
sample_info_df <- read.table(input_f,sep="\t",header=T)
head(sample_info_df)

,Sample_ID,Survival_days,Status_Cox,kmean_k5_mark_short,kmean_k4_mark_short,tumor_to_immune_mark,CD4_T_01_FYB1_Percent,CD4_T_02_CCL5_Percent,CD4_T_03_MT_ND4L_Percent,CD4_T_04_ISG15_Percent,⋯,WBC,Neutrophile_Count,Platelet_Count,BM_Blast_Percent,Survival_Status,Status_ggsurvfit,DiseaseClass,Source,BM_Blast_Percent_Mark1,BM_Blast_Percent_Mark2
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,AML_Beat_12_00051,707,1,Primitive,CD8T_Int_CD4T_Low,Primitive_CD8T_Int_CD4T_Low,undirection_P>0.05,depleted_P<=0.01,undirection_P>0.05,undirection_P>0.05,⋯,179,10,19,88,Dead,2,AML,Beat,80-90,80-90
2,AML_Beat_12_00211,1396,0,Mature,CD8T_Int_CD4T_Low,Mature_CD8T_Int_CD4T_Low,depleted_P<=0.001,expanded_P<=0.05,depleted_P<=0.001,expanded_P<=0.01,⋯,841,20,8,90,Alive,1,AML,Beat,90-100,90-100
3,AML_Beat_13_00157,1379,0,Primitive,CD8T_Int_CD4T_Low,Primitive_CD8T_Int_CD4T_Low,depleted_P<=0.001,expanded_P<=0.05,depleted_P<=0.001,expanded_P<=0.01,⋯,539,230,228,80,Alive,1,AML,Beat,80-90,80-90
4,AML_Beat_13_00163,400,0,GMP,CD8_Int_CD4_High,GMP_CD8_Int_CD4_High,undirection_P>0.05,undirection_P>0.05,undirection_P>0.05,expanded_P<=0.05,⋯,138,160,53,50,Alive,1,AML,Beat,50-60,50-60
5,AML_Beat_13_00195,972,0,Primitive,CD8T_Int_CD4T_Low,Primitive_CD8T_Int_CD4T_Low,undirection_P>0.05,depleted_P<=0.01,undirection_P>0.05,undirection_P>0.05,⋯,280,10,122,90,Alive,1,AML,Beat,90-100,90-100
6,AML_Beat_13_00232,1001,0,GMP,CD8T_Int_CD4T_Low,GMP_CD8T_Int_CD4T_Low,undirection_P>0.05,depleted_P<=0.01,undirection_P>0.05,undirection_P>0.05,⋯,294,240,34,55,Alive,1,AML,Beat,50-60,50-60


In [7]:
colnames(sample_info_df)

[1] "Sample_ID"                   "Survival_days"              
 [3] "Status_Cox"                  "kmean_k5_mark_short"        
 [5] "kmean_k4_mark_short"         "tumor_to_immune_mark"       
 [7] "CD4_T_01_FYB1_Percent"       "CD4_T_02_CCL5_Percent"      
 [9] "CD4_T_03_MT_ND4L_Percent"    "CD4_T_04_ISG15_Percent"     
[11] "CD4_T_05_APOOL_Percent"      "CD4_Treg_01_ITGB1_Percent"  
[13] "CD8_T_01_NBEAL1_Percent"     "CD8_T_02_LMNA_Percent"      
[15] "CD8_T_03_HBA2_Percent"       "CD8_T_04_CD8B_Percent"      
[17] "CD8_T_05_CCL4_Percent"       "CD8_T_06_CCR7_Percent"      
[19] "CD8_T_07_S100A11_Percent"    "CD8_T_08_ATP5MC2_Percent"   
[21] "CD8_T_09_MT_ND2_Percent"     "CD8_T_10_EEF1G_PCNP_Percent"
[23] "B_01_MS4A1_HLA_DRA_Percent"  "B_03_HLA_DRA_Percent"       
[25] "B_05_STMN1_Percent"          "B_06_DNTT_Percent"          
[27] "B_07_MS4A1_CD79A_Percent"    "NK_01_IL7R_Percent"         
[29] "NK_02_MT_CYB_Percent"        "NK_03_GZMK_Percent"         
[31] "NK_04_GZMH_Percent"          "NK_05_CD3D_Percent"         
[33] "NK_06_SPON2_Percent"         "Risk_Group"                 
[35] "IPSS"                        "Cytogenetic_Risk"           
[37] "WHO_category"                "Class_FAB"                  
[39] "Class_ELN"                   "Age"                        
[41] "Gender"                      "Haemoglobin"                
[43] "WBC"                         "Neutrophile_Count"          
[45] "Platelet_Count"              "BM_Blast_Percent"           
[47] "Survival_Status"             "Status_ggsurvfit"           
[49] "DiseaseClass"                "Source"                     
[51] "BM_Blast_Percent_Mark1"      "BM_Blast_Percent_Mark2"

In [8]:
## remove NA
sample_info_df = sample_info_df[!is.na(sample_info_df$BM_Blast_Percent_Mark2),]

In [9]:
## remove NA
sample_info_df = sample_info_df[sample_info_df$BM_Blast_Percent_Mark2 != "",]

In [10]:
dim(sample_info_df)

[1] 420  52

In [11]:
sample_info_df[,"Survival_days"]=sample_info_df[,"Survival_days"]/365

In [12]:
sample_info_df$kmean_k5_mark_short = factor(sample_info_df$`kmean_k5_mark_short`, levels = c("GMP","Mature","Prog","Primitive","intermediate"))

In [13]:
sample_info_df$kmean_k4_mark_short = factor(sample_info_df$`kmean_k4_mark_short`, levels = c("CD8T_High","CD8T_Int_CD4T_Low","CD8_Int_CD4_High","CD8T_Low"))

In [14]:
sample_info_df$tumor_to_immune_mark = factor(sample_info_df$tumor_to_immune_mark, levels = c("Prog_CD8T_High",
    "GMP_CD8T_High",
    "Prog_CD8_Int_CD4_High",
    "intermediate_CD8T_High",
    "Mature_CD8T_High",
    "Mature_CD8_Int_CD4_High",
    "Mature_CD8T_Int_CD4T_Low",
    "GMP_CD8T_Int_CD4T_Low",
    "intermediate_CD8T_Int_CD4T_Low",
    "GMP_CD8_Int_CD4_High",
    "GMP_CD8T_Low",
    "intermediate_CD8T_Low",
    "intermediate_CD8_Int_CD4_High",
    "Primitive_CD8T_Low",
    "Prog_CD8T_Low",
    "Primitive_CD8T_Int_CD4T_Low",
    "Mature_CD8T_Low",
    "Primitive_CD8_Int_CD4_High",
    "Prog_CD8T_Int_CD4T_Low",
    "Primitive_CD8T_High"))

In [15]:
table(sample_info_df$tumor_to_immune_mark)


                Prog_CD8T_High                  GMP_CD8T_High 
                             7                             16 
         Prog_CD8_Int_CD4_High         intermediate_CD8T_High 
                            95                              0 
              Mature_CD8T_High        Mature_CD8_Int_CD4_High 
                            31                              7 
      Mature_CD8T_Int_CD4T_Low          GMP_CD8T_Int_CD4T_Low 
                            33                             36 
intermediate_CD8T_Int_CD4T_Low           GMP_CD8_Int_CD4_High 
                            18                             12 
                  GMP_CD8T_Low          intermediate_CD8T_Low 
                            15                             38 
 intermediate_CD8_Int_CD4_High             Primitive_CD8T_Low 
                             5                             14 
                 Prog_CD8T_Low    Primitive_CD8T_Int_CD4T_Low 
                             3                        

In [16]:
sample_info_df$tumor_to_immune_mark =  droplevels(sample_info_df$tumor_to_immune_mark)

In [17]:
rt = sample_info_df[,c("Sample_ID","Survival_days","Status_Cox","tumor_to_immune_mark")]
head(rt)

,Sample_ID,Survival_days,Status_Cox,tumor_to_immune_mark
,<chr>,<dbl>,<int>,<fct>
1,AML_Beat_12_00051,1.936986,1,Primitive_CD8T_Int_CD4T_Low
2,AML_Beat_12_00211,3.824658,0,Mature_CD8T_Int_CD4T_Low
3,AML_Beat_13_00157,3.778082,0,Primitive_CD8T_Int_CD4T_Low
4,AML_Beat_13_00163,1.095890,0,GMP_CD8_Int_CD4_High
5,AML_Beat_13_00195,2.663014,0,Primitive_CD8T_Int_CD4T_Low
6,AML_Beat_13_00232,2.742466,0,GMP_CD8T_Int_CD4T_Low


In [19]:
rownames(rt) = rt$Sample_ID

In [18]:
# 1）subset

In [9]:
{
library(caret)
intrain_first<-createDataPartition(y=rt$Status_Cox,p=0.5,list=F)
train_first<-rt[intrain_first,]
test_first<-rt[-intrain_first,]
train_firstOut=cbind(id=row.names(train_first),train_first) 
test_firstOut=cbind(id=row.names(test_first),test_first)
}

In [10]:
table(train_first$tumor_to_immune_mark)


                Prog_CD8T_High                  GMP_CD8T_High 
                             2                              7 
         Prog_CD8_Int_CD4_High               Mature_CD8T_High 
                            48                             13 
       Mature_CD8_Int_CD4_High       Mature_CD8T_Int_CD4T_Low 
                             2                             18 
         GMP_CD8T_Int_CD4T_Low intermediate_CD8T_Int_CD4T_Low 
                            17                              9 
          GMP_CD8_Int_CD4_High                   GMP_CD8T_Low 
                             6                              9 
         intermediate_CD8T_Low  intermediate_CD8_Int_CD4_High 
                            20                              3 
            Primitive_CD8T_Low                  Prog_CD8T_Low 
                             7                              2 
   Primitive_CD8T_Int_CD4T_Low                Mature_CD8T_Low 
                            23                        

In [11]:
train_first$tumor_to_immune_mark =  droplevels(train_first$tumor_to_immune_mark)
test_first$tumor_to_immune_mark =  droplevels(test_first$tumor_to_immune_mark)

,Sample_ID,Survival_days,Status_Cox,tumor_to_immune_mark
,<chr>,<dbl>,<int>,<fct>
AML_Beat_13_00163,AML_Beat_13_00163,1.09589041,0,GMP_CD8_Int_CD4_High
AML_Beat_13_00232,AML_Beat_13_00232,2.74246575,0,GMP_CD8T_Int_CD4T_Low
AML_Beat_13_00658,AML_Beat_13_00658,0.55068493,0,GMP_CD8T_Int_CD4T_Low
AML_Beat_14_00034,AML_Beat_14_00034,2.48219178,0,Mature_CD8T_Int_CD4T_Low
AML_Beat_14_00228,AML_Beat_14_00228,2.15890411,0,Primitive_CD8T_Low
AML_Beat_14_00464,AML_Beat_14_00464,1.24109589,0,Primitive_CD8T_Int_CD4T_Low
AML_Beat_14_00787,AML_Beat_14_00787,1.21095890,0,Mature_CD8T_Int_CD4T_Low
AML_Beat_15_00377,AML_Beat_15_00377,1.55890411,0,GMP_CD8T_Int_CD4T_Low
AML_Beat_15_00680,AML_Beat_15_00680,1.22465753,0,Primitive_CD8T_Int_CD4T_Low


In [12]:
head(train_first)

,Sample_ID,Survival_days,Status_Cox,tumor_to_immune_mark
,<chr>,<dbl>,<int>,<fct>
AML_Beat_12_00051,AML_Beat_12_00051,1.9369863,1,Primitive_CD8T_Int_CD4T_Low
AML_Beat_13_00157,AML_Beat_13_00157,3.7780822,0,Primitive_CD8T_Int_CD4T_Low
AML_Beat_13_00195,AML_Beat_13_00195,2.6630137,0,Primitive_CD8T_Int_CD4T_Low
AML_Beat_13_00487,AML_Beat_13_00487,0.9808219,0,Primitive_CD8T_Int_CD4T_Low
AML_Beat_13_00513,AML_Beat_13_00513,0.6739726,1,Primitive_CD8T_Int_CD4T_Low
AML_Beat_14_00001,AML_Beat_14_00001,0.5945205,0,Mature_CD8T_Low


In [13]:
dim(train_first)

[1] 210   4

In [14]:
dim(test_first)

[1] 210   4

## 1）Univariate Cox

In [24]:
input_df = train_first

pFilterAll = 0.99
pFilterSignificant = 0.05

outTabAll = data.frame()
outTabSignificant = data.frame()

for (i in colnames(input_df)[4]) {
  print(i)
  
  if (is.numeric(input_df[, i])) {
    # 对于数值型，直接进行Cox回归
    cox <- coxph(Surv(Survival_days, Status_Cox) ~ input_df[, i], data = input_df)
    coxSummary = summary(cox)
    coxP = coxSummary$coefficients[, "Pr(>|z|)"]
    
    # 将结果添加到outTab中
    outTabAll = rbind(outTabAll,
                      cbind(id = i,
                            coef=coxSummary$coefficients[,"coef"],
                            HR = coxSummary$conf.int[, "exp(coef)"],
                            HR.95L = coxSummary$conf.int[, "lower .95"],
                            HR.95H = coxSummary$conf.int[, "upper .95"],
                            pvalue = coxP)
    )
    
    if (coxP[1] < pFilterSignificant) {
      outTabSignificant = rbind(outTabSignificant,
                                cbind(id = i,
                                      coef=coxSummary$coefficients[,"coef"],
                                      HR = coxSummary$conf.int[, "exp(coef)"],
                                      HR.95L = coxSummary$conf.int[, "lower .95"],
                                      HR.95H = coxSummary$conf.int[, "upper .95"],
                                      pvalue = coxP)
      )
    }
  } else if (is.factor(input_df[, i])) { # 1
    levels_i = levels(input_df[, i])
    control_group = levels_i[1]  

    cox <- coxph(Surv(Survival_days, Status_Cox) ~ input_df[, i], data = input_df)
    coxSummary = summary(cox)
    coxP = coxSummary$coefficients[, "Pr(>|z|)"]
    if (length(coxP) >= 1) {
      for (j in 1:length(coxP)) {
        if (coxP[j] < pFilterAll) {
          outTabAll = rbind(outTabAll,
                            cbind(id = paste(control_group, "_vs_", levels_i[j+1], sep = ""),
                                  coef=as.numeric(coxSummary$coefficients[,"coef"][j]),
                                  HR = coxSummary$conf.int[j, "exp(coef)"],
                                  HR.95L = coxSummary$conf.int[j, "lower .95"],
                                  HR.95H = coxSummary$conf.int[j, "upper .95"],
                                  pvalue = coxP[j])
          )
        }
        print(paste(control_group, "_vs_", levels_i[j+1], sep = ""))
        if (coxP[j] < pFilterSignificant) {
          outTabSignificant = rbind(outTabSignificant,
                                    cbind(id = paste(control_group, "_vs_", levels_i[j+1], sep = ""),
                                          coef=coxSummary$coefficients[,"coef"][j],
                                          HR = coxSummary$conf.int[j, "exp(coef)"],
                                          HR.95L = coxSummary$conf.int[j, "lower .95"],
                                          HR.95H = coxSummary$conf.int[j, "upper .95"],
                                          pvalue = coxP[j])
          )
        }
      }
    } 

  } # 1
}

[1] "tumor_to_immune_mark"
[1] "Prog_CD8T_High_vs_GMP_CD8T_High"
[1] "Prog_CD8T_High_vs_Prog_CD8_Int_CD4_High"
[1] "Prog_CD8T_High_vs_Mature_CD8T_High"
[1] "Prog_CD8T_High_vs_Mature_CD8_Int_CD4_High"
[1] "Prog_CD8T_High_vs_Mature_CD8T_Int_CD4T_Low"
[1] "Prog_CD8T_High_vs_GMP_CD8T_Int_CD4T_Low"
[1] "Prog_CD8T_High_vs_intermediate_CD8T_Int_CD4T_Low"
[1] "Prog_CD8T_High_vs_GMP_CD8_Int_CD4_High"
[1] "Prog_CD8T_High_vs_GMP_CD8T_Low"
[1] "Prog_CD8T_High_vs_intermediate_CD8T_Low"
[1] "Prog_CD8T_High_vs_intermediate_CD8_Int_CD4_High"
[1] "Prog_CD8T_High_vs_Primitive_CD8T_Low"
[1] "Prog_CD8T_High_vs_Prog_CD8T_Low"
[1] "Prog_CD8T_High_vs_Primitive_CD8T_Int_CD4T_Low"
[1] "Prog_CD8T_High_vs_Mature_CD8T_Low"
[1] "Prog_CD8T_High_vs_Primitive_CD8_Int_CD4_High"
[1] "Prog_CD8T_High_vs_Prog_CD8T_Int_CD4T_Low"
[1] "Prog_CD8T_High_vs_Primitive_CD8T_High"


In [25]:
outTabAll

,id,coef,HR,HR.95L,HR.95H,pvalue
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"input_df[, i]GMP_CD8T_High",Prog_CD8T_High_vs_GMP_CD8T_High,-1.48548265739391,0.226393044025544,0.014020154473341,3.65572365701174,0.295270004926605
"input_df[, i]Prog_CD8_Int_CD4_High",Prog_CD8T_High_vs_Prog_CD8_Int_CD4_High,-0.177915296768994,0.837013318218668,0.111127151441954,6.30441153025837,0.862888103809499
"input_df[, i]Mature_CD8T_High",Prog_CD8T_High_vs_Mature_CD8T_High,-0.155522393347399,0.855967908727568,0.0984347266379017,7.443318895644,0.887921824364944
"input_df[, i]Mature_CD8_Int_CD4_High",Prog_CD8T_High_vs_Mature_CD8_Int_CD4_High,1.66197537781477,5.26971023839443,0.569645327527211,48.7493614968913,0.143141562121789
"input_df[, i]Mature_CD8T_Int_CD4T_Low",Prog_CD8T_High_vs_Mature_CD8T_Int_CD4T_Low,0.946456225463178,2.57656270421139,0.324854857489271,20.4358199229092,0.370367870118028
"input_df[, i]GMP_CD8T_Int_CD4T_Low",Prog_CD8T_High_vs_GMP_CD8T_Int_CD4T_Low,0.132909270091187,1.14214636679402,0.141163984655232,9.24101374983695,0.900843678452482
"input_df[, i]intermediate_CD8T_Int_CD4T_Low",Prog_CD8T_High_vs_intermediate_CD8T_Int_CD4T_Low,1.66371325559496,5.27887631317766,0.64832729677444,42.9822024592669,0.119960863959584
"input_df[, i]GMP_CD8_Int_CD4_High",Prog_CD8T_High_vs_GMP_CD8_Int_CD4_High,0.688035633541772,1.98980298941539,0.218165717850211,18.1482039236101,0.541832403460717
"input_df[, i]GMP_CD8T_Low",Prog_CD8T_High_vs_GMP_CD8T_Low,0.732490818080628,2.08025569839966,0.239232115858068,18.0889750324812,0.506823908765208


In [26]:
outTabSignificant

,id,coef,HR,HR.95L,HR.95H,pvalue
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"input_df[, i]intermediate_CD8T_Low",Prog_CD8T_High_vs_intermediate_CD8T_Low,3.14781786040688,23.2851975501386,3.00795444331807,180.255530848692,0.00257294703459886
"input_df[, i]Primitive_CD8T_Low",Prog_CD8T_High_vs_Primitive_CD8T_Low,2.88278980429393,17.8640409042493,2.20108676166069,144.98472435858,0.00696586573271154


In [27]:
output_f = file.path(output_dir,"20250717_2_1_one_cox_result_all.xls")
write.table(outTabAll,file=output_f,sep="\t",row.names=F,quote=F)

output_f = file.path(output_dir,"20250717_2_1_one_cox_result_sign.xls")
write.table(outTabSignificant,file=output_f,sep="\t",row.names=F,quote=F)

## 2）Lasso regression

In [63]:
sigGenes = colnames(rt)
sigGenes

[1] "Sample_ID"            "Survival_days"        "Status_Cox"          
[4] "tumor_to_immune_mark"

In [64]:
sigGenes

[1] "Sample_ID"            "Survival_days"        "Status_Cox"          
[4] "tumor_to_immune_mark"

## 3）Construct a multi-factor Cox model

In [28]:
head(rt)

,Sample_ID,Survival_days,Status_Cox,tumor_to_immune_mark
,<chr>,<dbl>,<int>,<fct>
AML_Beat_12_00051,AML_Beat_12_00051,1.936986,1,Primitive_CD8T_Int_CD4T_Low
AML_Beat_12_00211,AML_Beat_12_00211,3.824658,0,Mature_CD8T_Int_CD4T_Low
AML_Beat_13_00157,AML_Beat_13_00157,3.778082,0,Primitive_CD8T_Int_CD4T_Low
AML_Beat_13_00163,AML_Beat_13_00163,1.095890,0,GMP_CD8_Int_CD4_High
AML_Beat_13_00195,AML_Beat_13_00195,2.663014,0,Primitive_CD8T_Int_CD4T_Low
AML_Beat_13_00232,AML_Beat_13_00232,2.742466,0,GMP_CD8T_Int_CD4T_Low


In [29]:
for(i in 1:1000){
	print(paste0("###### START : ",as.character(i)," #########"))
## 1） Divide the training set and the test set
library(caret)
intrain<-createDataPartition(y=rt$Status_Cox,p=0.7,list=F)
train<-rt[intrain,]
test<-rt[-intrain,]
trainOut=cbind(id=row.names(train),train)
testOut=cbind(id=row.names(test),test)

library(dplyr)
train <- train %>% select(-Sample_ID)

## 4） Build a Cox model
	  multiCox <- coxph(Surv(Survival_days, Status_Cox) ~ ., data = train)
	  multiCoxSum=summary(multiCox)

		#输出模型相关信息
		outMultiTab=data.frame()
		outMultiTab=cbind(
		               coef=multiCoxSum$coefficients[,"coef"],
		               HR=multiCoxSum$conf.int[,"exp(coef)"],
		               HR.95L=multiCoxSum$conf.int[,"lower .95"],
		               HR.95H=multiCoxSum$conf.int[,"upper .95"],
		               pvalue=multiCoxSum$coefficients[,"Pr(>|z|)"])
		outMultiTab=cbind(id=row.names(multiCoxSum$coefficients),outMultiTab)

## 5） ROC
		#train
		riskScore=predict(multiCox,type="risk",newdata=train)


		# Multivariable 
		# Extract the formula of the final optimized model
		final_formula <- formula(multiCox)
		outCol =  all.vars(final_formula)
		
		# Risk value grouping
		mediantrainRisk=median(riskScore)
		risk=as.vector(ifelse(riskScore>mediantrainRisk,"high","low"))
		trainRiskOut=cbind(id=trainOut$Sample_ID,cbind(train[,outCol],riskScore,risk))
		
		# test
		riskScoreTest=predict(multiCox,type="risk",newdata=test)
		riskTest=as.vector(ifelse(riskScoreTest>mediantrainRisk,"high","low"))
		testRiskOut=cbind(id=testOut$Sample_ID,cbind(test[,outCol],riskScore=riskScoreTest,risk=riskTest));         
		
		diff=survdiff(Surv(Survival_days, Status_Cox) ~risk,data = trainRiskOut)
		pValue=1-pchisq(diff$chisq,df=1)
		roc = survivalROC(Stime=trainRiskOut$Survival_days, status=trainRiskOut$Status_Cox, marker = riskScore, predict.time =5,method="KM") # 默认为5year生存期
		
		diffTest=survdiff(Surv(Survival_days, Status_Cox) ~riskTest,data = testRiskOut)
		pValueTest=1-pchisq(diffTest$chisq,df=1)
		rocTest = survivalROC(Stime=testRiskOut$Survival_days, status=testRiskOut$Status_Cox, marker = riskScoreTest, predict.time =5,  method="KM") # 默认为5year生存期

# ## 运行检查
if((pValue<0.05) & (roc$AUC>0.8) & (pValueTest<0.05) & (rocTest$AUC>0.8)){

## 1）Divide the training set and the test set
output_f_path_train = file.path(output_dir,"20250717_1_Risk_Model_Build_1307_sample_train_294.xls")
output_f_path_test = file.path(output_dir,"20250717_1_Risk_Model_Build_1307_sample_test_126.xls")
write.table(trainOut,file=output_f_path_train,sep="\t",quote=F,row.names=F)
write.table(testOut,file=output_f_path_test,sep="\t",quote=F,row.names=F)


## 4）Build a Cox model
output_f = file.path(output_dir,"20250720_1_Risk_Model_Build_1307_sample_train_294_multi_cox_result.xls")
write.table(outMultiTab,file=output_f,sep="\t",row.names=F,quote=F)

output_f = file.path(output_dir,"20250720_1_Risk_Model_Build_1307_sample_train_294_multi_cox_result_risktrain.xls")
write.table(trainRiskOut,file=output_f,sep="\t",row.names=F,quote=F)

output_f = file.path(output_dir,"20250720_1_Risk_Model_Build_1307_sample_train_294_multi_cox_result_riskTest.xls")
write.table(testRiskOut,file=output_f,sep="\t",row.names=F,quote=F)

	print(paste0("###### Save : ",as.character(i)," #########"))
	print(paste0("pValue : ",as.character(pValue)))
	print(paste0("pValueTest : ",as.character(pValueTest)))
	print(paste0("roc$AUC : ",as.character(roc$AUC)))
	print(paste0("rocTest$AUC : ",as.character(rocTest$AUC)))
	print(paste0("###### Save : ",as.character(i)," #########"))

    break
}
	print(paste0("pValue : ",as.character(pValue)))
	print(paste0("pValueTest : ",as.character(pValueTest)))
	print(paste0("roc$AUC : ",as.character(roc$AUC)))
	print(paste0("rocTest$AUC : ",as.character(rocTest$AUC)))

	print(paste0("###### Done : ",as.character(i)," #########"))
	print(paste0("######                                                    #########"))
}

[1] "###### START : 1 #########"



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


[1] "pValue : 0"
[1] "pValueTest : 3.88874470402811e-08"
[1] "roc$AUC : 0.7899304340973"
[1] "rocTest$AUC : 0.83516356003664"
[1] "###### Done : 1 #########"
[1] "######                                                    #########"
[1] "###### START : 2 #########"
[1] "pValue : 0"
[1] "pValueTest : 3.03391397523667e-05"
[1] "roc$AUC : 0.787155701140675"
[1] "rocTest$AUC : 0.810803383990392"
[1] "###### Done : 2 #########"
[1] "######                                                    #########"
[1] "###### START : 3 #########"
[1] "pValue : 1.765254609154e-14"
[1] "pValueTest : 1.98639549253699e-10"
[1] "roc$AUC : 0.816784748042128"
[1] "rocTest$AUC : 0.783377876491582"
[1] "###### Done : 3 #########"
[1] "######                                                    #########"
[1] "###### START : 4 #########"


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


[1] "pValue : 0"
[1] "pValueTest : 0.000190961155905001"
[1] "roc$AUC : 0.800374248340302"
[1] "rocTest$AUC : 0.797666785687521"
[1] "###### Done : 4 #########"
[1] "######                                                    #########"
[1] "###### START : 5 #########"


...

[1] "pValue : 0"
[1] "pValueTest : 3.05126141797452e-07"
[1] "roc$AUC : 0.800746797648018"
[1] "rocTest$AUC : 0.78556517579162"
[1] "###### Done : 60 #########"
[1] "######                                                    #########"
[1] "###### START : 61 #########"
[1] "pValue : 0"
[1] "pValueTest : 2.50636331990961e-05"
[1] "roc$AUC : 0.870176498704596"
[1] "rocTest$AUC : 0.586298559822078"
[1] "###### Done : 61 #########"
[1] "######                                                    #########"
[1] "###### START : 62 #########"


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


[1] "pValue : 5.55111512312578e-16"
[1] "pValueTest : 7.56898708154097e-08"
[1] "roc$AUC : 0.737226545773485"
[1] "rocTest$AUC : 0.831839391876351"
[1] "###### Done : 62 #########"
[1] "######                                                    #########"
[1] "###### START : 63 #########"


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  18 ; coefficient may be infinite. ”


[1] "pValue : 7.7715611723761e-16"
[1] "pValueTest : 7.63674612436205e-09"
[1] "roc$AUC : 0.789313299450562"
[1] "rocTest$AUC : 0.776803228733052"
[1] "###### Done : 63 #########"
[1] "######                                                    #########"
[1] "###### START : 64 #########"
[1] "pValue : 0"
[1] "pValueTest : 7.74171359291254e-05"
[1] "roc$AUC : 0.841997263130709"
[1] "rocTest$AUC : 0.741710967436869"
[1] "###### Done : 64 #########"
[1] "######                                                    #########"
[1] "###### START : 65 #########"
[1] "###### Save : 65 #########"
[1] "pValue : 0"
[1] "pValueTest : 6.97753531619938e-08"
[1] "roc$AUC : 0.814997063921848"
[1] "rocTest$AUC : 0.817272279894914"
[1] "###### Save : 65 #########"


## 4）Calculate the AUC value

In [30]:
print(paste0("pValue : ",as.character(pValue)))
print(paste0("pValueTest : ",as.character(pValueTest)))
print(paste0("roc$AUC : ",as.character(roc$AUC)))
print(paste0("rocTest$AUC : ",as.character(rocTest$AUC)))

print(paste0("###### Done : ",as.character(i)," #########"))
print(paste0("######                           #########"))

[1] "pValue : 0"
[1] "pValueTest : 6.97753531619938e-08"
[1] "roc$AUC : 0.814997063921848"
[1] "rocTest$AUC : 0.817272279894914"


[1] "###### Done : 65 #########"
[1] "######                           #########"


In [9]:
multiCoxSum$coefficients[, "coef"]

tumor_to_immune_markGMP_CD8T_High 
                                       -1.20905127 
         tumor_to_immune_markProg_CD8_Int_CD4_High 
                                       -0.04752712 
              tumor_to_immune_markMature_CD8T_High 
                                       -0.40472407 
       tumor_to_immune_markMature_CD8_Int_CD4_High 
                                        1.46421710 
      tumor_to_immune_markMature_CD8T_Int_CD4T_Low 
                                        0.81665218 
         tumor_to_immune_markGMP_CD8T_Int_CD4T_Low 
                                        0.26292309 
tumor_to_immune_markintermediate_CD8T_Int_CD4T_Low 
                                        1.26966246 
          tumor_to_immune_markGMP_CD8_Int_CD4_High 
                                       -0.08240109 
                  tumor_to_immune_markGMP_CD8T_Low 
                                        1.31327116 
         tumor_to_immune_markintermediate_CD8T_Low 
                                        2.56429255 
 tumor_to_immune_markintermediate_CD8_Int_CD4_High 
                                        1.05896337 
            tumor_to_immune_markPrimitive_CD8T_Low 
                                        2.79308732 
                 tumor_to_immune_markProg_CD8T_Low 
                                        0.55550560 
   tumor_to_immune_markPrimitive_CD8T_Int_CD4T_Low 
                                        0.49870347 
               tumor_to_immune_markMature_CD8T_Low 
                                        1.73944998 
    tumor_to_immune_markPrimitive_CD8_Int_CD4_High 
                                        1.69170115 
        tumor_to_immune_markProg_CD8T_Int_CD4T_Low 
                                        0.66042205 
           tumor_to_immune_markPrimitive_CD8T_High 
                                        1.88537770

In [7]:
final_formula

Surv(Survival_days, Status_Cox) ~ tumor_to_immune_mark

In [6]:
mediantrainRisk

[1] 1.646585

## 5）forest map

In [31]:
options(forestplot_new_page = FALSE)
output_f = file.path(output_dir,"20250717_5_Risk_Model_Build_1307_sample_train_294_multi_cox_result_HR_forest.pdf")

pdf(file=output_f,width = 12,height = 12)
ggforest(multiCox,main = "Hazard ratio",cpositions = c(0.02,0.22, 0.4), fontsize = 0.7, refLabel = "reference", noDigits = 2)
dev.off()

Warning message in .get_data(model, data = data):
“The `data` argument is not provided. Data will be extracted from model fit.”


png 
  2

In [113]:
multiCox

Call:
coxph(formula = Surv(Survival_days, Status_Cox) ~ ., data = train)

                                                       coef exp(coef) se(coef)
tumor_to_immune_markGMP_CD8T_High                  -1.20905   0.29848  1.41616
tumor_to_immune_markProg_CD8_Int_CD4_High          -0.04753   0.95358  1.02328
tumor_to_immune_markMature_CD8T_High               -0.40472   0.66716  1.09772
tumor_to_immune_markMature_CD8_Int_CD4_High         1.46422   4.32416  1.16168
tumor_to_immune_markMature_CD8T_Int_CD4T_Low        0.81665   2.26291  1.04423
tumor_to_immune_markGMP_CD8T_Int_CD4T_Low           0.26292   1.30073  1.05598
tumor_to_immune_markintermediate_CD8T_Int_CD4T_Low  1.26966   3.55965  1.07543
tumor_to_immune_markGMP_CD8_Int_CD4_High           -0.08240   0.92090  1.22694
tumor_to_immune_markGMP_CD8T_Low                    1.31327   3.71832  1.07759
tumor_to_immune_markintermediate_CD8T_Low           2.56429  12.99146  1.03385
tumor_to_immune_markintermediate_CD8_Int_CD4_High   1.058

In [ ]:
# 存在无穷大的值

## 6）KM curve - Training set

In [169]:
if(TRUE){
  # 根据kmean进行分组
  trainRiskOut$risk = factor(trainRiskOut$risk,levels = c("low","high"))

  
  fit_Group <- survfit(Surv(Survival_days,Status_Cox) ~ risk, data = trainRiskOut) # 数据集来源
  
  #summary(fit_Group)
  
  # surv_object =
  surv_object = ggsurvplot(fit_Group, data = trainRiskOut,  
                           surv.median.line = "hv",
                           conf.int = FALSE,
                           conf.int.alpha = 0.1,
                           risk.table = TRUE,
                           align = "hv",
                           pval = TRUE,
                           palette = c("#2671b2","#d52331"), 
                           title = paste0("Union(20) Model Risk Group train(n=294)"),  #
                           xlab = "Follow up time(d)",
                           legend = "right", #
                           legend.title = "Group", 
                           break.x.by = 1,  #
                           risk.table.y.text.col = TRUE, #
                           risk.table.x.text.angle = 0,  #
                           risk.table.height = 0.2,     #
                           font.main = c(8, "bold", "black"),  #
  )
  
  library(patchwork)
  plot_object = surv_object$plot / surv_object$table +
    plot_layout(heights = c(5, 1))  
  
  output_f_path = file.path(output_dir,"20250717_6_1_Risk_Model_Build_1307_sample_train_294_Survial.pdf")
  ggsave(filename=output_f_path,plot=plot_object,device=pdf,width=180,height=190,units="mm")
  
}

## 6）KM curve - test set

In [170]:
if(TRUE){
   testRiskOut$risk = factor( testRiskOut$risk,levels = c("low","high"))

  fit_Group <- survfit(Surv(Survival_days,Status_Cox) ~ risk, data =  testRiskOut) # 数据集来源
  
  #summary(fit_Group)
  
  # surv_object =
  surv_object = ggsurvplot(fit_Group, data =  testRiskOut,  
                           surv.median.line = "hv",
                           conf.int = FALSE,
                           conf.int.alpha = 0.1,
                           risk.table = TRUE,
                           align = "hv",
                           pval = TRUE,
                           palette = c("#2671b2","#d52331"), 
                           title = paste0("Union(20) Model Risk Group Test(n=126)"),  
                           xlab = "Follow up time(d)",
                           legend = "right", 
                           legend.title = "Group", 
                           break.x.by = 1, 
                           risk.table.y.text.col = TRUE, 
                           risk.table.x.text.angle = 0, 
                           risk.table.height = 0.2,
                           font.main = c(8, "bold", "black")
  )
  
  library(patchwork)
  plot_object = surv_object$plot / surv_object$table +
  plot_layout(heights = c(5, 1))

  output_f_path = file.path(output_dir,"20250717_6_2_Risk_Model_Build_1307_sample_test_126_Survial.pdf")
  ggsave(filename=output_f_path,plot=plot_object,device=pdf,width=180,height=190,units="mm") # 默认180*180
  
}

## 7）## ROC plot

In [8]:
roc_1 = survivalROC(Stime=trainRiskOut$Survival_days, status=trainRiskOut$Status_Cox, marker = riskScore, predict.time =1,method="KM") # 默认为5year生存期
roc_3 = survivalROC(Stime=trainRiskOut$Survival_days, status=trainRiskOut$Status_Cox, marker = riskScore, predict.time =3,method="KM") # 默认为5year生存期
roc_5 = survivalROC(Stime=trainRiskOut$Survival_days, status=trainRiskOut$Status_Cox, marker = riskScore, predict.time =5,method="KM") # 默认为5year生存期

In [10]:
roc_05 = survivalROC(Stime=trainRiskOut$Survival_days, status=trainRiskOut$Status_Cox, marker = riskScore, predict.time =0.5,method="KM") # 默认为5year生存期
roc_2 = survivalROC(Stime=trainRiskOut$Survival_days, status=trainRiskOut$Status_Cox, marker = riskScore, predict.time =2,method="KM") # 默认为5year生存期
roc_4 = survivalROC(Stime=trainRiskOut$Survival_days, status=trainRiskOut$Status_Cox, marker = riskScore, predict.time =4,method="KM") # 默认为5year生存期

In [9]:
roc_1

$cut.values
 [1]       -Inf  0.2984803  0.6671609  0.9209025  0.9535846  1.0000000
 [7]  1.3007267  1.6465850  1.7428219  1.9356091  2.2629113  2.8833804
[13]  3.5596509  3.7183171  4.3241565  5.4287079  5.6942106  6.5888426
[19] 12.9914643 16.3313623

$TP
 [1] 1.00000000 0.98453540 0.96709666 0.95890294 0.91243874 0.89715584
 [7] 0.82096087 0.75831941 0.74674251 0.74253863 0.66885924 0.65865038
[13] 0.59921526 0.56595556 0.54605730 0.51260215 0.31986610 0.30913134
[19] 0.09628076 0.00000000

$FP
 [1] 1.00000000 0.95235258 0.84079509 0.79973215 0.47230559 0.45948211
 [7] 0.36534072 0.28981591 0.28523874 0.27223758 0.19188436 0.18168251
[13] 0.14954558 0.10021968 0.08954772 0.07522158 0.01545954 0.01547674
[19] 0.00498695 0.00000000

$predict.time
[1] 1

$Survival
[1] 0.6820523

$AUC
[1] 0.8204625

In [10]:
rocTest_1 = survivalROC(Stime=testRiskOut$Survival_days, status=testRiskOut$Status_Cox, marker = riskScoreTest, predict.time =1,  method="KM") # 默认为5year生存期
rocTest_3 = survivalROC(Stime=testRiskOut$Survival_days, status=testRiskOut$Status_Cox, marker = riskScoreTest, predict.time =3,  method="KM") # 默认为5year生存期
rocTest_5 = survivalROC(Stime=testRiskOut$Survival_days, status=testRiskOut$Status_Cox, marker = riskScoreTest, predict.time =5,  method="KM") # 默认为5year生存期

In [11]:
rocTest_05 = survivalROC(Stime=testRiskOut$Survival_days, status=testRiskOut$Status_Cox, marker = riskScoreTest, predict.time =0.5,  method="KM") # 默认为5year生存期
rocTest_2 = survivalROC(Stime=testRiskOut$Survival_days, status=testRiskOut$Status_Cox, marker = riskScoreTest, predict.time =2,  method="KM") # 默认为5year生存期
rocTest_4 = survivalROC(Stime=testRiskOut$Survival_days, status=testRiskOut$Status_Cox, marker = riskScoreTest, predict.time =4,  method="KM") # 默认为5year生存期

In [192]:
## 1）training set
rocCol=rainbow(3)
aucText=c()

output_f = file.path(output_dir,"20250720_1_1_Risk_Model_Build_1307_sample_train_294_multi_ROC_train_result.pdf")
pdf(file=output_f,width=6,height=6)
par(oma=c(0.5,1,0,1),font.lab=1.5,font.axis=1.5)

# ## 1) training set 1 Year 

plot(smooth_roc,type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[1], 
  xlab="False positive rate", ylab="True positive rate",
  main=paste("Union(20) Model train(n=294) : ROC curve"),
  lwd = 2, cex.main=1.3, cex.lab=1.2, cex.axis=1.2, font=1.2)
aucText=c(aucText,paste0("train(n=294) Union(20) 1 year "," (AUC=",sprintf("%.3f",roc_1$AUC),")"))
abline(0,1)

## 2) training set 3 Year 
aucText=c(aucText,paste0("train(n=294) Union(20) 3 year "," (AUC=",sprintf("%.3f",roc_3$AUC),")"))
lines(roc_3$FP, roc_3$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[2],lwd = 2)

## 3) training set 5 Year 
aucText=c(aucText,paste0("train(n=294) Union(20) 5 year "," (AUC=",sprintf("%.3f",roc_5$AUC),")"))
lines(roc_5$FP, roc_5$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[3],lwd = 2)

legend("bottomright", aucText,lwd=1,bty="n",col=rocCol,cex=0.5)
dev.off()

png 
  2

In [172]:
## 1）training set
rocCol=rainbow(3)
aucText=c()

output_f = file.path(output_dir,"20250720_1_Risk_Model_Build_1307_sample_train_294_multi_rocTest_Test_result.pdf")
pdf(file=output_f,width=6,height=6)
par(oma=c(0.5,1,0,1),font.lab=1.5,font.axis=1.5)

# ## 1) training set 1 Year 

plot(rocTest_1$FP, rocTest_1$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[1], 
  xlab="False positive rate", ylab="True positive rate",
  main=paste("Union(20) Model Test(n=126) : ROC curve"),
  lwd = 2, cex.main=1.3, cex.lab=1.2, cex.axis=1.2, font=1.2)
aucText=c(aucText,paste0("Test(n=126) Union(20) 1 year "," (AUC=",sprintf("%.3f",rocTest_1$AUC),")"))
abline(0,1)


## 2) training set 3 Year 
aucText=c(aucText,paste0("Test(n=126) Union(20) 3 year "," (AUC=",sprintf("%.3f",rocTest_3$AUC),")"))
lines(rocTest_3$FP, rocTest_3$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[2],lwd = 2)

## 3) training set 5 Year 
aucText=c(aucText,paste0("Test(n=126) Union(20) 5 year "," (AUC=",sprintf("%.3f",rocTest_5$AUC),")"))
lines(rocTest_5$FP, rocTest_5$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[3],lwd = 2)


legend("bottomright", aucText,lwd=1,bty="n",col=rocCol,cex=0.5)
dev.off()

png 
  2

## 8) Split the samples according to the training set and test set of the model. Sample information table

In [38]:
sample_info_df_train =sample_info_df[sample_info_df$Sample_ID %in% trainOut$Sample_ID,]
sample_info_df_test =sample_info_df[sample_info_df$Sample_ID %in% testOut$Sample_ID,]

In [13]:
print(dim(sample_info_df_train))
print(dim(sample_info_df_test))

[1] 294  52
[1] 126  52


In [40]:
head(sample_info_df_train)

,Sample_ID,Survival_days,Status_Cox,kmean_k5_mark_short,kmean_k4_mark_short,tumor_to_immune_mark,CD4_T_01_FYB1_Percent,CD4_T_02_CCL5_Percent,CD4_T_03_MT_ND4L_Percent,CD4_T_04_ISG15_Percent,⋯,WBC,Neutrophile_Count,Platelet_Count,BM_Blast_Percent,Survival_Status,Status_ggsurvfit,DiseaseClass,Source,BM_Blast_Percent_Mark1,BM_Blast_Percent_Mark2
,<chr>,<dbl>,<int>,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,AML_Beat_12_00051,1.936986,1,Primitive,CD8T_Int_CD4T_Low,Primitive_CD8T_Int_CD4T_Low,undirection_P>0.05,depleted_P<=0.01,undirection_P>0.05,undirection_P>0.05,⋯,179,10,19,88,Dead,2,AML,Beat,80-90,80-90
2,AML_Beat_12_00211,3.824658,0,Mature,CD8T_Int_CD4T_Low,Mature_CD8T_Int_CD4T_Low,depleted_P<=0.001,expanded_P<=0.05,depleted_P<=0.001,expanded_P<=0.01,⋯,841,20,8,90,Alive,1,AML,Beat,90-100,90-100
3,AML_Beat_13_00157,3.778082,0,Primitive,CD8T_Int_CD4T_Low,Primitive_CD8T_Int_CD4T_Low,depleted_P<=0.001,expanded_P<=0.05,depleted_P<=0.001,expanded_P<=0.01,⋯,539,230,228,80,Alive,1,AML,Beat,80-90,80-90
4,AML_Beat_13_00163,1.095890,0,GMP,CD8_Int_CD4_High,GMP_CD8_Int_CD4_High,undirection_P>0.05,undirection_P>0.05,undirection_P>0.05,expanded_P<=0.05,⋯,138,160,53,50,Alive,1,AML,Beat,50-60,50-60
6,AML_Beat_13_00232,2.742466,0,GMP,CD8T_Int_CD4T_Low,GMP_CD8T_Int_CD4T_Low,undirection_P>0.05,depleted_P<=0.01,undirection_P>0.05,undirection_P>0.05,⋯,294,240,34,55,Alive,1,AML,Beat,50-60,50-60
7,AML_Beat_13_00245,2.347945,0,intermediate,CD8T_Int_CD4T_Low,intermediate_CD8T_Int_CD4T_Low,undirection_P>0.05,depleted_P<=0.01,undirection_P>0.05,undirection_P>0.05,⋯,195,200,109,30,Alive,1,AML,Beat,30-40,30-40


## 8-2) Split the samples according to the training set and test set of the model. Sample information table—— 5:5
# Used for some clinical indicators

In [16]:
sample_info_df_train_first =sample_info_df[sample_info_df$Sample_ID %in% train_first$Sample_ID,]
sample_info_df_test_first =sample_info_df[sample_info_df$Sample_ID %in% test_first$Sample_ID,]

In [12]:
print(dim(sample_info_df_train_first))
print(dim(sample_info_df_test_first))

[1] 210  52
[1] 210  52


In [18]:
head(sample_info_df_train_first)

,Sample_ID,Survival_days,Status_Cox,kmean_k5_mark_short,kmean_k4_mark_short,tumor_to_immune_mark,CD4_T_01_FYB1_Percent,CD4_T_02_CCL5_Percent,CD4_T_03_MT_ND4L_Percent,CD4_T_04_ISG15_Percent,⋯,WBC,Neutrophile_Count,Platelet_Count,BM_Blast_Percent,Survival_Status,Status_ggsurvfit,DiseaseClass,Source,BM_Blast_Percent_Mark1,BM_Blast_Percent_Mark2
,<chr>,<dbl>,<int>,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,AML_Beat_12_00051,1.936986,1,Primitive,CD8T_Int_CD4T_Low,Primitive_CD8T_Int_CD4T_Low,undirection_P>0.05,depleted_P<=0.01,undirection_P>0.05,undirection_P>0.05,⋯,179,10,19,88,Dead,2,AML,Beat,80-90,80-90
2,AML_Beat_12_00211,3.824658,0,Mature,CD8T_Int_CD4T_Low,Mature_CD8T_Int_CD4T_Low,depleted_P<=0.001,expanded_P<=0.05,depleted_P<=0.001,expanded_P<=0.01,⋯,841,20,8,90,Alive,1,AML,Beat,90-100,90-100
3,AML_Beat_13_00157,3.778082,0,Primitive,CD8T_Int_CD4T_Low,Primitive_CD8T_Int_CD4T_Low,depleted_P<=0.001,expanded_P<=0.05,depleted_P<=0.001,expanded_P<=0.01,⋯,539,230,228,80,Alive,1,AML,Beat,80-90,80-90
5,AML_Beat_13_00195,2.663014,0,Primitive,CD8T_Int_CD4T_Low,Primitive_CD8T_Int_CD4T_Low,undirection_P>0.05,depleted_P<=0.01,undirection_P>0.05,undirection_P>0.05,⋯,280,10,122,90,Alive,1,AML,Beat,90-100,90-100
7,AML_Beat_13_00245,2.347945,0,intermediate,CD8T_Int_CD4T_Low,intermediate_CD8T_Int_CD4T_Low,undirection_P>0.05,depleted_P<=0.01,undirection_P>0.05,undirection_P>0.05,⋯,195,200,109,30,Alive,1,AML,Beat,30-40,30-40
8,AML_Beat_13_00393,3.389041,1,intermediate,CD8T_Int_CD4T_Low,intermediate_CD8T_Int_CD4T_Low,depleted_P<=0.001,expanded_P<=0.05,depleted_P<=0.001,expanded_P<=0.01,⋯,342,230,22,21,Dead,2,AML,Beat,20-30,20-30


## 9) Constructing Functions

In [14]:
getROC <- function(trainOut, testOut, var_str) {
  formula_str <- paste("Surv(Survival_days, Status_Cox) ~", var_str)
  formula <- as.formula(formula_str)

  multiCox <- coxph(formula, data = trainOut)
  
  multiCoxSum <- summary(multiCox)
  
  outMultiTab <- data.frame()
  outMultiTab <- cbind(
    coef = multiCoxSum$coefficients[, "coef"],
    HR = multiCoxSum$conf.int[, "exp(coef)"],
    HR.95L = multiCoxSum$conf.int[, "lower .95"],
    HR.95H = multiCoxSum$conf.int[, "upper .95"],
    pvalue = multiCoxSum$coefficients[, "Pr(>|z|)"]
  )
  
  outMultiTab <- cbind(id = row.names(multiCoxSum$coefficients), outMultiTab)
  
  riskScore <- predict(multiCox, type = "risk", newdata = trainOut)
  
  final_formula <- formula(multiCox)
  outCol <- all.vars(final_formula)
  
  mediantrainRisk <- median(riskScore)
  risk <- as.vector(ifelse(riskScore > mediantrainRisk, "high", "low"))
  
  trainRiskOut <- cbind(id = trainOut$Sample_ID, cbind(trainOut[, outCol], riskScore, risk))
  
  riskScoreTest <- predict(multiCox, type = "risk", newdata = testOut)
  riskTest <- as.vector(ifelse(riskScoreTest > mediantrainRisk, "high", "low"))
  
  testRiskOut <- cbind(id = testOut$Sample_ID, cbind(testOut[, outCol], riskScore = riskScoreTest, risk = riskTest))
  
  roc_05 <- survivalROC(Stime = trainRiskOut$Survival_days, status = trainRiskOut$Status_Cox, 
                     marker = riskScore, predict.time = 0.5, method = "KM")
  roc_1 <- survivalROC(Stime = trainRiskOut$Survival_days, status = trainRiskOut$Status_Cox, 
                     marker = riskScore, predict.time = 1, method = "KM")
  roc_2 <- survivalROC(Stime = trainRiskOut$Survival_days, status = trainRiskOut$Status_Cox, 
                     marker = riskScore, predict.time = 2, method = "KM")
  roc_3 <- survivalROC(Stime = trainRiskOut$Survival_days, status = trainRiskOut$Status_Cox, 
                    marker = riskScore, predict.time = 3, method = "KM")
  roc_4 <- survivalROC(Stime = trainRiskOut$Survival_days, status = trainRiskOut$Status_Cox, 
                    marker = riskScore, predict.time = 4, method = "KM")                  
  roc_5 <- survivalROC(Stime = trainRiskOut$Survival_days, status = trainRiskOut$Status_Cox, 
                    marker = riskScore, predict.time = 5, method = "KM")

  rocTest_05 <- survivalROC(Stime = testRiskOut$Survival_days, status = testRiskOut$Status_Cox, 
                         marker = riskScoreTest, predict.time = 0.5, method = "KM")
  rocTest_1 <- survivalROC(Stime = testRiskOut$Survival_days, status = testRiskOut$Status_Cox, 
                         marker = riskScoreTest, predict.time = 1, method = "KM")
  rocTest_2 <- survivalROC(Stime = testRiskOut$Survival_days, status = testRiskOut$Status_Cox, 
                         marker = riskScoreTest, predict.time = 2, method = "KM")
  rocTest_3 <- survivalROC(Stime = testRiskOut$Survival_days, status = testRiskOut$Status_Cox, 
                         marker = riskScoreTest, predict.time = 3, method = "KM")
  rocTest_4 <- survivalROC(Stime = testRiskOut$Survival_days, status = testRiskOut$Status_Cox, 
                         marker = riskScoreTest, predict.time = 4, method = "KM")
  rocTest_5 <- survivalROC(Stime = testRiskOut$Survival_days, status = testRiskOut$Status_Cox, 
                         marker = riskScoreTest, predict.time = 5, method = "KM")


  # 输出结果到文件
  output_f <- file.path(output_dir,paste0(var_str,"_multi_cox_result.xls"))
  write.table(outMultiTab, file = output_f, sep = "\t", row.names = F, quote = F)
  
  output_f <- file.path(output_dir, paste0(var_str,"_multi_cox_result_risktrain.xls"))
  write.table(trainRiskOut, file = output_f, sep = "\t", row.names = F, quote = F)
  
  output_f <- file.path(output_dir, paste0(var_str,"_multi_cox_result_riskTest.xls"))
  write.table(testRiskOut, file = output_f, sep = "\t", row.names = F, quote = F)
  
  # Return ROC and ROCTest
  return(list(roc_05 = roc_05,roc_1 = roc_1,roc_2=roc_2,roc_3=roc_3,roc_4=roc_4,roc_5=roc_5,
  rocTest_05 = rocTest_05,rocTest_1 = rocTest_1,rocTest_2 = rocTest_2,rocTest_3=rocTest_3,
  rocTest_4=rocTest_4,rocTest_5=rocTest_5))
}


## 10) Tumor kmean group Risk Group

In [19]:
dim(sample_info_df_train)

[1] 294  52

In [20]:
result_tumor <- getROC(sample_info_df_train, sample_info_df_test, "kmean_k5_mark_short")
roc_1_tumor <- result_tumor$roc_1
roc_3_tumor <- result_tumor$roc_3
roc_5_tumor <- result_tumor$roc_5
roc_05_tumor <- result_tumor$roc_05
roc_2_tumor <- result_tumor$roc_2
roc_4_tumor <- result_tumor$roc_4


rocTest_1_tumor <- result_tumor$rocTest_1
rocTest_3_tumor <- result_tumor$rocTest_3
rocTest_5_tumor <- result_tumor$rocTest_5
rocTest_05_tumor <- result_tumor$rocTest_05
rocTest_2_tumor <- result_tumor$rocTest_2
rocTest_4_tumor <- result_tumor$rocTest_4

In [21]:
roc_1_tumor

$cut.values
[1]      -Inf 0.8020602 1.0000000 1.7888319 2.7594463 4.7185336

$TP
[1] 1.0000000 0.9283851 0.7945638 0.4897103 0.2819193 0.0000000

$FP
[1] 1.00000000 0.63941522 0.40756777 0.14573632 0.05309673 0.00000000

$predict.time
[1] 1

$Survival
[1] 0.6820523

$AUC
[1] 0.7587617

In [22]:
roc_3_tumor

$cut.values
[1]      -Inf 0.8020602 1.0000000 1.7888319 2.7594463 4.7185336

$TP
[1] 1.0000000 0.8241258 0.6469384 0.3499876 0.1824717 0.0000000

$FP
[1] 1.00000000 0.61848522 0.38925736 0.13980090 0.05704636 0.00000000

$predict.time
[1] 3

$Survival
[1] 0.4514347

$AUC
[1] 0.6681512

## 11）immune kmean group

In [23]:
result_immune <- getROC(sample_info_df_train, sample_info_df_test, "kmean_k4_mark_short")
roc_1_immune <- result_immune$roc_1
roc_3_immune <- result_immune$roc_3
roc_5_immune <- result_immune$roc_5
roc_05_immune <- result_immune$roc_05
roc_2_immune <- result_immune$roc_2
roc_4_immune <- result_immune$roc_4


rocTest_1_immune <- result_immune$rocTest_1
rocTest_3_immune <- result_immune$rocTest_3
rocTest_5_immune <- result_immune$rocTest_5
rocTest_05_immune <- result_immune$rocTest_05
rocTest_2_immune <- result_immune$rocTest_2
rocTest_4_immune <- result_immune$rocTest_4

In [24]:
roc_1_immune

$cut.values
[1]     -Inf 1.000000 1.802430 2.784292 9.720386

$TP
[1] 1.0000000 0.9386990 0.8215253 0.5431193 0.0000000

$FP
[1] 1.0000000 0.8290983 0.4249210 0.1308129 0.0000000

$predict.time
[1] 1

$Survival
[1] 0.6820523

$AUC
[1] 0.7575849

## 14) Blast factor

In [19]:
sample_info_df_train_Blast_factor = sample_info_df_train_first[!is.na(sample_info_df_train_first$BM_Blast_Percent_Mark2) & sample_info_df_train_first$BM_Blast_Percent_Mark2 != "", ]
sample_info_df_test_Blast_factor = sample_info_df_test_first[!is.na(sample_info_df_test_first$BM_Blast_Percent_Mark2) & sample_info_df_test_first$BM_Blast_Percent_Mark2 != "", ]

In [20]:
print(dim(sample_info_df_train_Blast_factor))
print(dim(sample_info_df_test_Blast_factor))

[1] 294  52
[1] 126  52


In [27]:
table(sample_info_df_train_Blast_factor$BM_Blast_Percent_Mark2)


   0-2  10-20    2-5  20-30  30-40  40-50   5-10  50-60  60-70  70-80  80-90 
    18     18     31      6     14     13     19     15      9     21     24 
90-100 
    22 

In [28]:
sample_info_df_train_Blast_factor$BM_Blast_Percent_Mark2 = factor(sample_info_df_train_Blast_factor$BM_Blast_Percent_Mark2,levels = c(
    "0-2",
    '2-5',
    '5-10',
    '10-20',
    "20-30",
    "30-40",
    "40-50",
    "50-60",
    "60-70",
    "70-80",
    "80-90",
    "90-100" 
))

In [29]:
sample_info_df_test_Blast_factor$BM_Blast_Percent_Mark2 = factor(sample_info_df_test_Blast_factor$BM_Blast_Percent_Mark2,levels = c(
    "0-2",
    '2-5',
    '5-10',
    '10-20',
    "20-30",
    "30-40",
    "40-50",
    "50-60",
    "60-70",
    "70-80",
    "80-90",
    "90-100" 
))

In [25]:
result_Blast_factor <- getROC(sample_info_df_train_Blast_factor, sample_info_df_test_Blast_factor, "BM_Blast_Percent_Mark2")
roc_1_Blast_factor <- result_Blast_factor$roc_1
roc_3_Blast_factor <- result_Blast_factor$roc_3
roc_5_Blast_factor <- result_Blast_factor$roc_5
roc_05_Blast_factor <- result_Blast_factor$roc_05
roc_2_Blast_factor <- result_Blast_factor$roc_2
roc_4_Blast_factor <- result_Blast_factor$roc_4


rocTest_1_Blast_factor <- result_Blast_factor$rocTest_1
rocTest_3_Blast_factor <- result_Blast_factor$rocTest_3
rocTest_5_Blast_factor <- result_Blast_factor$rocTest_5
rocTest_05_Blast_factor <- result_Blast_factor$rocTest_05
rocTest_2_Blast_factor <- result_Blast_factor$rocTest_2
rocTest_4_Blast_factor <- result_Blast_factor$rocTest_4

In [26]:
roc_1_Blast_factor

$cut.values
 [1]      -Inf  1.000000  2.163835  4.366761  4.569744  6.397083  6.878258
 [8]  8.318626  9.011266 11.423829 11.765980 14.959866 18.404748

$TP
 [1] 1.0000000 0.9988233 0.9711885 0.9510965 0.9020583 0.7748598 0.6115557
 [8] 0.4993506 0.4386596 0.3499027 0.3128466 0.1415997 0.0000000

$FP
 [1] 1.00000000 0.87091127 0.66173090 0.54234722 0.43061007 0.33735988
 [7] 0.24822449 0.19771744 0.16401047 0.10867786 0.08445971 0.02101498
[13] 0.00000000

$predict.time
[1] 1

$Survival
[1] 0.6609042

$AUC
[1] 0.7829192

## 15) IPSS

In [21]:
sample_info_df_train_IPSS = sample_info_df_train_first[!is.na(sample_info_df_train_first$IPSS) & sample_info_df_train_first$IPSS != "", ]
sample_info_df_test_IPSS  = sample_info_df_test_first[!is.na(sample_info_df_test_first$IPSS)   & sample_info_df_test_first$IPSS != "", ]

In [22]:
print(dim(sample_info_df_train_IPSS))
print(dim(sample_info_df_test_IPSS))

[1] 61 52
[1] 35 52


In [47]:
table(sample_info_df_train_IPSS$IPSS)


 High Int-1 Int-2   Low 
    2    19    12    13 

In [49]:
# IPSS rename
sample_info_df_train_IPSS$IPSS = gsub("Int-1", "Int", sample_info_df_train_IPSS$IPSS)
sample_info_df_train_IPSS$IPSS = gsub("Int-2", "Int", sample_info_df_train_IPSS$IPSS)

In [50]:
sample_info_df_test_IPSS$IPSS = gsub("Int-1", "Int", sample_info_df_test_IPSS$IPSS)
sample_info_df_test_IPSS$IPSS = gsub("Int-2", "Int", sample_info_df_test_IPSS$IPSS)

In [51]:
table(sample_info_df_train_IPSS$IPSS)


High  Int  Low 
   2   31   13 

In [69]:
table(sample_info_df_test_IPSS$IPSS)


 Low  Int High 
  18   28    4 

In [52]:
sample_info_df_train_IPSS$IPSS = factor(sample_info_df_train_IPSS$IPSS,levels = c("Low","Int","High"))

In [53]:
sample_info_df_test_IPSS$IPSS = factor(sample_info_df_test_IPSS$IPSS,levels = c("Low","Int","High"))

In [27]:
result_IPSS <- getROC(sample_info_df_train_IPSS, sample_info_df_test_IPSS, "IPSS")
roc_1_IPSS <- result_IPSS$roc_1
roc_3_IPSS <- result_IPSS$roc_3
roc_5_IPSS <- result_IPSS$roc_5
roc_05_IPSS <- result_IPSS$roc_05
roc_2_IPSS <- result_IPSS$roc_2
roc_4_IPSS <- result_IPSS$roc_4


rocTest_1_IPSS <- result_IPSS$rocTest_1
rocTest_3_IPSS <- result_IPSS$rocTest_3
rocTest_5_IPSS <- result_IPSS$rocTest_5
rocTest_05_IPSS <- result_IPSS$rocTest_05
rocTest_2_IPSS <- result_IPSS$rocTest_2
rocTest_4_IPSS <- result_IPSS$rocTest_4

In [28]:
roc_1_IPSS

$cut.values
[1]     -Inf 1.000000 1.503181 6.692704

$TP
[1] 1.000000 1.070394 0.000000 0.000000

$FP
[1] 1.00000000 0.69370496 0.04639563 0.00000000

$predict.time
[1] 1

$Survival
[1] 0.9371197

$AUC
[1] 0.6635138

In [67]:
roc_3_IPSS

$cut.values
[1]     -Inf 1.000000 1.503181 6.692704

$TP
[1] 1.0000000 0.8842281 0.1290780 0.0000000

$FP
[1] 1.0000000 0.6326507 0.0000000 0.0000000

$predict.time
[1] 3

$Survival
[1] 0.6631628

$AUC
[1] 0.6666194

In [68]:
roc_5_IPSS

$cut.values
[1]     -Inf 1.000000 1.503181 6.692704

$TP
[1] 1.0000000 0.7959640 0.0732188 0.0000000

$FP
[1] 1.0000000 0.6025244 0.0000000 0.0000000

$predict.time
[1] 5

$Survival
[1] 0.4061872

$AUC
[1] 0.6187778

In [63]:
rocTest_1_IPSS

$cut.values
[1]     -Inf 1.000000 1.503181 6.692704

$TP
[1] 1.000000 1.080757 1.576104 0.000000

$FP
[1] 1.0000000 0.6164317 0.0000000 0.0000000

$predict.time
[1] 1

$Survival
[1] 0.9492419

$AUC
[1] 1.217943

In [64]:
rocTest_3_IPSS

$cut.values
[1]     -Inf 1.000000 1.503181 6.692704

$TP
[1] 1.0000000 0.7813175 0.3086790 0.0000000

$FP
[1] 1.0000000 0.5905621 0.0000000 0.0000000

$predict.time
[1] 3

$Survival
[1] 0.7408311

$AUC
[1] 0.6865248

In [65]:
rocTest_5_IPSS

$cut.values
[1]     -Inf 1.000000 1.503181 6.692704

$TP
[1] 1.0000000 0.8065134 0.2342720 0.0000000

$FP
[1] 1.000000 0.553652 0.000000 0.000000

$predict.time
[1] 5

$Survival
[1] 0.6585166

$AUC
[1] 0.6912833

## 16) WHO

In [56]:
sample_info_df_train_WHO = sample_info_df_train_first[!is.na(sample_info_df_train_first$WHO) & sample_info_df_train_first$WHO != "", ]
sample_info_df_test_WHO  = sample_info_df_test_first[!is.na(sample_info_df_test_first$WHO) & sample_info_df_test_first$WHO != "", ]

In [108]:
dim(sample_info_df_train_WHO)

[1] 86 53

In [109]:
dim(sample_info_df_test_WHO)

[1] 93 53

In [57]:
table(sample_info_df_train_WHO$WHO)


    5q-    CMML      RA    RAEB   RAEB1   RAEB2    RARS  RARS-T    RCMD RCMD-RS 
      3       4      12       4      14      14      19       2       9       5 

In [76]:
table(sample_info_df_test_WHO$WHO)


    5q-    CMML      RA    RAEB   RAEB1   RAEB2    RARS  RARS-T    RCMD RCMD-RS 
      3       4       9       2      12      20      21       2       7      13 

In [60]:
replace_vec <- c(
  "RA" = "RA",
  "CMML" = "RA",
  "5q-" = "RA",
  "RARS" = "RA",
  "RARS-T" = "RA",
  "RCMD" = "RCMD",
  "RCMD-RS" = "RCMD",
  "RAEB" = "RAEB",
  "RAEB1" = "RAEB",
  "RAEB2" = "RAEB"
)

In [80]:

# Obtain the value in the WHO_Mark column
who_mark_values <- sample_info_df_train_WHO$WHO

# Use the `match` function to find the matching indices
match_indices <- match(who_mark_values, names(replace_vec))

# Only when the match finds an effective match, will the replacement be carried out
sample_info_df_train_WHO$WHO_Mark[!is.na(match_indices)] <- replace_vec[match_indices[!is.na(match_indices)]]

In [81]:
sample_info_df_train_WHO$WHO_Mark

[1] "RA"   "RA"   "RA"   "RA"   "RA"   "RA"   "RA"   "RAEB" "RAEB" "RAEB"
[11] "RA"   "RA"   "RA"   "RA"   "RA"   "RA"   "RAEB" "RA"   "RA"   "RAEB"
[21] "RA"   "RAEB" "RA"   "RA"   "RA"   "RA"   "RAEB" "RA"   "RA"   "RA"  
[31] "RA"   "RA"   "RA"   "RA"   "RAEB" "RAEB" "RAEB" "RCMD" "RA"   "RAEB"
[41] "RAEB" "RAEB" "RAEB" "RAEB" "RA"   "RCMD" "RAEB" "RCMD" "RCMD" "RA"  
[51] "RA"   "RAEB" "RAEB" "RAEB" "RAEB" "RAEB" "RAEB" "RCMD" "RA"   "RAEB"
[61] "RCMD" "RCMD" "RAEB" "RA"   "RCMD" "RAEB" "RA"   "RA"   "RA"   "RAEB"
[71] "RAEB" "RA"   "RCMD" "RAEB" "RAEB" "RA"   "RCMD" "RAEB" "RAEB" "RA"  
[81] "RAEB" "RCMD" "RCMD" "RCMD" "RA"   "RCMD"

In [84]:
table(sample_info_df_train_WHO$WHO_Mark)


  RA RAEB RCMD 
  40   32   14 

In [85]:
table(sample_info_df_test_WHO$WHO_Mark)


  RA RAEB RCMD 
  39   34   20 

In [86]:
sample_info_df_train_WHO$WHO_Mark = factor(sample_info_df_train_WHO$WHO_Mark,levels = c("RA","RAEB","RCMD"))

In [87]:
sample_info_df_test_WHO$WHO_Mark = factor(sample_info_df_test_WHO$WHO_Mark,levels = c("RA","RAEB","RCMD"))

In [29]:
result_WHO <- getROC(sample_info_df_train_WHO, sample_info_df_test_WHO, "WHO_Mark")
roc_1_WHO <- result_WHO$roc_1
roc_3_WHO <- result_WHO$roc_3
roc_5_WHO <- result_WHO$roc_5
roc_05_WHO <- result_WHO$roc_05
roc_2_WHO <- result_WHO$roc_2
roc_4_WHO <- result_WHO$roc_4

rocTest_1_WHO <- result_WHO$rocTest_1
rocTest_3_WHO <- result_WHO$rocTest_3
rocTest_5_WHO <- result_WHO$rocTest_5
rocTest_05_WHO <- result_WHO$rocTest_05
rocTest_2_WHO <- result_WHO$rocTest_2
rocTest_4_WHO <- result_WHO$rocTest_4

In [30]:
roc_1_WHO

$cut.values
[1]     -Inf 1.000000 3.552370 3.624862

$TP
[1] 1.0000000 1.1175527 0.8032077 0.0000000

$FP
[1] 1.0000000 0.4967463 0.3438752 0.0000000

$predict.time
[1] 1

$Survival
[1] 0.9385679

$AUC
[1] 0.8177491

In [31]:
rocTest_1_WHO

$cut.values
[1]     -Inf 1.000000 3.552370 3.624862

$TP
[1] 1.0000000 0.8336923 0.9105810 0.0000000

$FP
[1] 1.0000000 0.5664991 0.3351249 0.0000000

$predict.time
[1] 1

$Survival
[1] 0.9470568

$AUC
[1] 0.7518227

## 17) Cytogenetic_Risk

In [95]:
sample_info_df_train_Cytogenetic_Risk = sample_info_df_train_first[!is.na(sample_info_df_train_first$Cytogenetic_Risk) & sample_info_df_train_first$Cytogenetic_Risk != "", ]
sample_info_df_test_Cytogenetic_Risk  = sample_info_df_test_first[!is.na(sample_info_df_test_first$Cytogenetic_Risk) & sample_info_df_test_first$Cytogenetic_Risk != "", ]

In [96]:
print(dim(sample_info_df_train_Cytogenetic_Risk))
print(dim(sample_info_df_test_Cytogenetic_Risk))

[1] 121  52
[1] 129  52


In [154]:
table(sample_info_df_Cytogenetic_Risk$Cytogenetic_Risk)


Good  Int Poor 
  82  129   39 

In [97]:
sample_info_df_train_Cytogenetic_Risk$Cytogenetic_Risk = factor(sample_info_df_train_Cytogenetic_Risk$Cytogenetic_Risk,levels = c("Good","Int","Poor"))
sample_info_df_test_Cytogenetic_Risk$Cytogenetic_Risk  = factor(sample_info_df_test_Cytogenetic_Risk$Cytogenetic_Risk,levels = c("Good","Int","Poor"))

In [32]:
result_Cyto <- getROC(sample_info_df_train_Cytogenetic_Risk, sample_info_df_test_Cytogenetic_Risk, "Cytogenetic_Risk")
roc_1_Cyto <- result_Cyto$roc_1
roc_3_Cyto <- result_Cyto$roc_3
roc_5_Cyto <- result_Cyto$roc_5
roc_05_Cyto <- result_Cyto$roc_05
roc_2_Cyto <- result_Cyto$roc_2
roc_4_Cyto <- result_Cyto$roc_4

rocTest_1_Cyto <- result_Cyto$rocTest_1
rocTest_3_Cyto <- result_Cyto$rocTest_3
rocTest_5_Cyto <- result_Cyto$rocTest_5
rocTest_05_Cyto <- result_Cyto$rocTest_05
rocTest_2_Cyto <- result_Cyto$rocTest_2
rocTest_4_Cyto <- result_Cyto$rocTest_4

In [33]:
rocTest_1_Cyto

$cut.values
[1]     -Inf 1.000000 1.703943 6.385216

$TP
[1] 1.0000000 1.0060379 0.3152989 0.0000000

$FP
[1] 1.0000000 0.5511406 0.1052815 0.0000000

$predict.time
[1] 1

$Survival
[1] 0.7630804

$AUC
[1] 0.7613771

## 18) Class_ELN

In [100]:
sample_info_df_train_Class_ELN = sample_info_df_train_first[!is.na(sample_info_df_train_first$Class_ELN) & sample_info_df_train_first$Class_ELN != "", ]
sample_info_df_test_Class_ELN  = sample_info_df_test_first[!is.na(sample_info_df_test_first$Class_ELN) & sample_info_df_test_first$Class_ELN != "", ]

In [101]:
print(dim(sample_info_df_train_Class_ELN))
print(dim(sample_info_df_test_Class_ELN))

[1] 33 52
[1] 34 52


In [160]:
table(sample_info_df_Class_ELN$Class_ELN)


     Adverse    Favorable Intermediate 
          17           35           15 

In [102]:
sample_info_df_train_Class_ELN$Class_ELN = factor(sample_info_df_train_Class_ELN$Class_ELN,levels = c("Favorable","Intermediate","Adverse"))
sample_info_df_test_Class_ELN$Class_ELN  = factor(sample_info_df_test_Class_ELN$Class_ELN,levels = c("Favorable","Intermediate","Adverse"))

In [34]:
result_ELN <- getROC(sample_info_df_train_Class_ELN, sample_info_df_test_Class_ELN, "Class_ELN")
roc_1_ELN <- result_Class_ELN$roc_1
roc_3_ELN <- result_Class_ELN$roc_3
roc_5_ELN <- result_Class_ELN$roc_5
roc_05_ELN <- result_Class_ELN$roc_05
roc_2_ELN <- result_Class_ELN$roc_2
roc_4_ELN <- result_Class_ELN$roc_4

rocTest_1_ELN <- result_Class_ELN$rocTest_1
rocTest_3_ELN <- result_Class_ELN$rocTest_3
rocTest_5_ELN <- result_Class_ELN$rocTest_5
rocTest_05_ELN <- result_Class_ELN$rocTest_05
rocTest_2_ELN <- result_Class_ELN$rocTest_2
rocTest_4_ELN <- result_Class_ELN$rocTest_4

In [35]:
rocTest_1_ELN

$cut.values
[1]     -Inf 1.000000 1.420486 1.681412

$TP
[1] 1.0000000 0.9253387 0.5099105 0.0000000

$FP
[1] 1.0000000 0.5215804 0.2653380 0.0000000

$predict.time
[1] 1

$Survival
[1] 0.762069

$AUC
[1] 0.712095

In [10]:
roc_1_ELN

$cut.values
[1]     -Inf 1.000000 3.238698 3.653981

$TP
[1] 1.0000000 0.7659697 0.4396764 0.0000000

$FP
[1] 1.0000000 0.4036202 0.2060184 0.0000000

$predict.time
[1] 1

$Survival
[1] 0.7958001

$AUC
[1] 0.691004

## 17) ROC merge

## train-1year

In [36]:
## 1）training set
rocCol=rainbow(8)
aucText=c()

output_f = file.path(output_dir,"20250720_5_1_modif_Risk_Model_Build_1307_sample_Train_294_multi_rocTrain_Train_result_Union20_1_Year_survail.pdf")
pdf(file=output_f,width=6,height=6)
par(oma=c(0.5,1,0,1),font.lab=1.5,font.axis=1.5)

# ## 1) training set
plot(roc_1$FP, roc_1$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[1], 
  xlab="False positive rate", ylab="True positive rate",
  main=paste("Train 1 Year"),
  lwd = 2, cex.main=1.3, cex.lab=1.2, cex.axis=1.2, font=1.2)
aucText=c(aucText,paste0("n=294 Union(20)"," AUC=",sprintf("%.3f",roc_1$AUC),""))
abline(0,1)


## 2) Tumor 
aucText=c(aucText,paste0("n=294 Tumor(5)"," AUC=",sprintf("%.3f",roc_1_tumor$AUC),""))
lines(roc_1_tumor$FP, roc_1_tumor$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[2],lwd = 2)


## 3) Immune 
aucText=c(aucText,paste0("n=294 Immune(4)"," AUC=",sprintf("%.3f",roc_1_immune$AUC),""))
lines(roc_1_immune$FP, roc_1_immune$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[3],lwd = 2)

## 4) Cyto
aucText=c(aucText,paste0("n=121 Cytogenetic Risk(3) 1 year"," AUC=",sprintf("%.3f",roc_1_Cyto$AUC),""))
lines(roc_1_Cyto$FP, roc_1_Cyto$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[4],lwd = 2)

## 5) IPSS 
aucText=c(aucText,paste0("n=46 IPSS(3) 1 year"," AUC=",sprintf("%.3f",roc_1_IPSS$AUC),""))
lines(roc_1_IPSS$FP, roc_1_IPSS$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[5],lwd = 2)

# 6) WHO 
aucText=c(aucText,paste0("n=86 WHO(3) 1 year"," AUC=",sprintf("%.3f",roc_1_WHO$AUC),""))
lines(roc_1_WHO$FP, roc_1_WHO$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[6],lwd = 2)


## 7) ELN
aucText=c(aucText,paste0("n=33 ENL2017(3) 1 year"," AUC=",sprintf("%.3f",roc_1_ELN$AUC),""))
lines(roc_1_ELN$FP, roc_1_ELN$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[7],lwd = 2)

legend("bottomright", aucText,lwd=1,bty="n",col=rocCol,cex=0.5)
dev.off()

png 
  2

## train-3Year

In [38]:
## 1）training set
rocCol=rainbow(8)
aucText=c()

output_f = file.path(output_dir,"20250720_3_1_Risk_Model_Build_1307_sample_Train_294_multi_rocTrain_Train_result_Union20_3_Year_survail.pdf")
pdf(file=output_f,width=6,height=6)
par(oma=c(0.5,1,0,1),font.lab=1.5,font.axis=1.5)

# ## 1) training set
plot(roc_3$FP, roc_3$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[1], 
  xlab="False positive rate", ylab="True positive rate",
  main=paste("Train 3 Year"),
  lwd = 2, cex.main=1.3, cex.lab=1.2, cex.axis=1.2, font=1.2)
aucText=c(aucText,paste0("n=294 Union(20)"," AUC=",sprintf("%.3f",roc_3$AUC),""))
abline(0,1)


## 2) Tumor 
aucText=c(aucText,paste0("n=294 Tumor(5)"," AUC=",sprintf("%.3f",roc_3_tumor$AUC),""))
lines(roc_3_tumor$FP, roc_3_tumor$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[2],lwd = 2)


## 3) Immune 
aucText=c(aucText,paste0("n=294 Immune(4)"," AUC=",sprintf("%.3f",roc_3_immune$AUC),""))
lines(roc_3_immune$FP, roc_3_immune$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[3],lwd = 2)

## 4) Cyto
aucText=c(aucText,paste0("n=121 Cytogenetic Risk(3) 3 year"," AUC=",sprintf("%.3f",roc_3_Cyto$AUC),""))
lines(roc_3_Cyto$FP, roc_3_Cyto$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[4],lwd = 2)

## 5) IPSS 
aucText=c(aucText,paste0("n=46 IPSS(3) 3 year"," AUC=",sprintf("%.3f",roc_3_IPSS$AUC),""))
lines(roc_3_IPSS$FP, roc_3_IPSS$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[5],lwd = 2)

# 6) WHO 
aucText=c(aucText,paste0("n=86 WHO(3) 3 year"," AUC=",sprintf("%.3f",roc_3_WHO$AUC),""))
lines(roc_3_WHO$FP, roc_3_WHO$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[6],lwd = 2)


## 7) ELN
aucText=c(aucText,paste0("n=33 ENL2017(3) 3 year"," AUC=",sprintf("%.3f",roc_3_ELN$AUC),""))
lines(roc_3_ELN$FP, roc_3_ELN$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[7],lwd = 2)

legend("bottomright", aucText,lwd=1,bty="n",col=rocCol,cex=0.5)
dev.off()

png 
  2

## train-5Year

In [39]:
## 1）training set
rocCol=rainbow(8)
aucText=c()

output_f = file.path(output_dir,"20250720_3_1_Risk_Model_Build_1307_sample_Train_294_multi_rocTrain_Train_result_Union20_5_Year_survail.pdf")
pdf(file=output_f,width=6,height=6)
par(oma=c(0.5,1,0,1),font.lab=1.5,font.axis=1.5)

# ## 1) training set
plot(roc_5$FP, roc_5$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[1], 
  xlab="False positive rate", ylab="True positive rate",
  main=paste("Train 5 year"),
  lwd = 2, cex.main=1.3, cex.lab=1.2, cex.axis=1.2, font=1.2)
aucText=c(aucText,paste0("n=294 Union(20)"," AUC=",sprintf("%.3f",roc_5$AUC),""))
abline(0,1)


## 2) Tumor 
aucText=c(aucText,paste0("n=294 Tumor(5)"," AUC=",sprintf("%.3f",roc_5_tumor$AUC),""))
lines(roc_5_tumor$FP, roc_5_tumor$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[2],lwd = 2)


## 3) Immune 
aucText=c(aucText,paste0("n=294 Immune(4)"," AUC=",sprintf("%.3f",roc_5_immune$AUC),""))
lines(roc_5_immune$FP, roc_5_immune$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[3],lwd = 2)

## 4) Cyto
aucText=c(aucText,paste0("n=121 Cytogenetic Risk(3) 5 year"," AUC=",sprintf("%.3f",roc_5_Cyto$AUC),""))
lines(roc_5_Cyto$FP, roc_5_Cyto$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[4],lwd = 2)

## 5) IPSS 
aucText=c(aucText,paste0("n=46 IPSS(3) 5 year"," AUC=",sprintf("%.3f",roc_5_IPSS$AUC),""))
lines(roc_5_IPSS$FP, roc_5_IPSS$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[5],lwd = 2)

# 6) WHO 
aucText=c(aucText,paste0("n=86 WHO(3) 5 year"," AUC=",sprintf("%.3f",roc_5_WHO$AUC),""))
lines(roc_5_WHO$FP, roc_5_WHO$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[6],lwd = 2)


## 7) ELN
aucText=c(aucText,paste0("n=33 ENL2017(3) 5 year"," AUC=",sprintf("%.3f",roc_5_ELN$AUC),""))
lines(roc_5_ELN$FP, roc_5_ELN$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[7],lwd = 2)

legend("bottomright", aucText,lwd=1,bty="n",col=rocCol,cex=0.5)
dev.off()

png 
  2

## train-05Year

In [59]:
## 1）training set
rocCol=rainbow(8)
aucText=c()

output_f = file.path(output_dir,"20250720_3_1_Risk_Model_Build_1307_sample_Train_294_multi_rocTrain_Train_result_Union20_0.5_Year_survail.pdf")
pdf(file=output_f,width=6,height=6)
par(oma=c(0.5,1,0,1),font.lab=1.5,font.axis=1.5)

# ## 1) training set
plot(roc_05$FP, roc_05$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[1], 
  xlab="False positive rate", ylab="True positive rate",
  main=paste("Train 0.5 year"),
  lwd = 2, cex.main=1.3, cex.lab=1.2, cex.axis=1.2, font=1.2)
aucText=c(aucText,paste0("n=294 Union(20)"," AUC=",sprintf("%.3f",roc_05$AUC),""))
abline(0,1)


## 2) Tumor 
aucText=c(aucText,paste0("n=294 Tumor(5)"," AUC=",sprintf("%.3f",roc_05_tumor$AUC),""))
lines(roc_05_tumor$FP, roc_05_tumor$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[2],lwd = 2)


## 3) Immune 
aucText=c(aucText,paste0("n=294 Immune(4)"," AUC=",sprintf("%.3f",roc_05_immune$AUC),""))
lines(roc_05_immune$FP, roc_05_immune$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[3],lwd = 2)

## 4) Cyto
aucText=c(aucText,paste0("n=121 Cytogenetic Risk(3) 5 year"," AUC=",sprintf("%.3f",roc_05_Cyto$AUC),""))
lines(roc_05_Cyto$FP, roc_05_Cyto$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[4],lwd = 2)

## 5) IPSS 
aucText=c(aucText,paste0("n=46 IPSS(3) 5 year"," AUC=",sprintf("%.3f",roc_05_IPSS$AUC),""))
lines(roc_05_IPSS$FP, roc_05_IPSS$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[5],lwd = 2)

# 6) WHO 
aucText=c(aucText,paste0("n=86 WHO(3) 5 year"," AUC=",sprintf("%.3f",roc_05_WHO$AUC),""))
lines(roc_05_WHO$FP, roc_05_WHO$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[6],lwd = 2)


## 7) ELN
aucText=c(aucText,paste0("n=33 ENL2017(3) 5 year"," AUC=",sprintf("%.3f",roc_05_ELN$AUC),""))
lines(roc_05_ELN$FP, roc_05_ELN$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[7],lwd = 2)


legend("bottomright", aucText,lwd=1,bty="n",col=rocCol,cex=0.5)
dev.off()

png 
  2

## train-2Year

In [47]:
## 1）training set
rocCol=rainbow(8)
aucText=c()

output_f = file.path(output_dir,"20250720_3_1_Risk_Model_Build_1307_sample_Train_294_multi_rocTrain_Train_result_Union20_2_Year_survail.pdf")
pdf(file=output_f,width=6,height=6)
par(oma=c(0.5,1,0,1),font.lab=1.5,font.axis=1.5)

# ## 1) training set
plot(roc_2$FP, roc_2$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[1], 
  xlab="False positive rate", ylab="True positive rate",
  main=paste("Train 2 year"),
  lwd = 2, cex.main=1.3, cex.lab=1.2, cex.axis=1.2, font=1.2)
aucText=c(aucText,paste0("n=294 Union(20)"," AUC=",sprintf("%.3f",roc_2$AUC),""))
abline(0,1)


## 2) Tumor 
aucText=c(aucText,paste0("n=294 Tumor(5)"," AUC=",sprintf("%.3f",roc_2_tumor$AUC),""))
lines(roc_2_tumor$FP, roc_2_tumor$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[2],lwd = 2)


## 3) Immune 
aucText=c(aucText,paste0("n=294 Immune(4)"," AUC=",sprintf("%.3f",roc_2_immune$AUC),""))
lines(roc_2_immune$FP, roc_2_immune$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[3],lwd = 2)

## 4) Cyto
aucText=c(aucText,paste0("n=121 Cytogenetic Risk(3) 5 year"," AUC=",sprintf("%.3f",roc_2_Cyto$AUC),""))
lines(roc_2_Cyto$FP, roc_2_Cyto$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[4],lwd = 2)

## 5) IPSS 
aucText=c(aucText,paste0("n=46 IPSS(3) 5 year"," AUC=",sprintf("%.3f",roc_2_IPSS$AUC),""))
lines(roc_2_IPSS$FP, roc_2_IPSS$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[5],lwd = 2)

# 6) WHO 
aucText=c(aucText,paste0("n=86 WHO(3) 5 year"," AUC=",sprintf("%.3f",roc_2_WHO$AUC),""))
lines(roc_2_WHO$FP, roc_2_WHO$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[6],lwd = 2)


## 7) ELN
aucText=c(aucText,paste0("n=33 ENL2017(3) 5 year"," AUC=",sprintf("%.3f",roc_2_ELN$AUC),""))
lines(roc_2_ELN$FP, roc_2_ELN$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[7],lwd = 2)

legend("bottomright", aucText,lwd=1,bty="n",col=rocCol,cex=0.5)
dev.off()

png 
  2

## train-4Year

In [53]:
## 1）training set
rocCol=rainbow(8)
aucText=c()

output_f = file.path(output_dir,"20250720_3_1_Risk_Model_Build_1307_sample_Train_294_multi_rocTrain_Train_result_Union20_4_Year_survail.pdf")
pdf(file=output_f,width=6,height=6)
par(oma=c(0.5,1,0,1),font.lab=1.5,font.axis=1.5)

# ## 1) training set
plot(roc_4$FP, roc_4$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[1], 
  xlab="False positive rate", ylab="True positive rate",
  main=paste("Train 4 year"),
  lwd = 2, cex.main=1.3, cex.lab=1.2, cex.axis=1.2, font=1.2)
aucText=c(aucText,paste0("n=294 Union(20)"," AUC=",sprintf("%.3f",roc_4$AUC),""))
abline(0,1)


## 2) Tumor 
aucText=c(aucText,paste0("n=294 Tumor(5)"," AUC=",sprintf("%.3f",roc_4_tumor$AUC),""))
lines(roc_4_tumor$FP, roc_4_tumor$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[2],lwd = 2)


## 3) Immune 
aucText=c(aucText,paste0("n=294 Immune(4)"," AUC=",sprintf("%.3f",roc_4_immune$AUC),""))
lines(roc_4_immune$FP, roc_4_immune$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[3],lwd = 2)

## 4) Cyto
aucText=c(aucText,paste0("n=121 Cytogenetic Risk(3) 5 year"," AUC=",sprintf("%.3f",roc_4_Cyto$AUC),""))
lines(roc_4_Cyto$FP, roc_4_Cyto$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[4],lwd = 2)

## 5) IPSS 
aucText=c(aucText,paste0("n=46 IPSS(3) 5 year"," AUC=",sprintf("%.3f",roc_4_IPSS$AUC),""))
lines(roc_4_IPSS$FP, roc_4_IPSS$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[5],lwd = 2)

# 6) WHO 
aucText=c(aucText,paste0("n=86 WHO(3) 5 year"," AUC=",sprintf("%.3f",roc_4_WHO$AUC),""))
lines(roc_4_WHO$FP, roc_4_WHO$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[6],lwd = 2)


## 7) ELN
aucText=c(aucText,paste0("n=33 ENL2017(3) 5 year"," AUC=",sprintf("%.3f",roc_4_ELN$AUC),""))
lines(roc_4_ELN$FP, roc_4_ELN$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocCol[7],lwd = 2)

legend("bottomright", aucText,lwd=1,bty="n",col=rocCol,cex=0.5)
dev.off()

png 
  2

## test set

## test-1year

In [40]:
## 1）training set
rocTestCol=rainbow(8)
aucText=c()

output_f = file.path(output_dir,"20250720_5_1_modif_Risk_Model_Build_1307_sample_Test_294_multi_rocTestTest_Test_result_Union20_1_Year_survail.pdf")
pdf(file=output_f,width=6,height=6)
par(oma=c(0.5,1,0,1),font.lab=1.5,font.axis=1.5)

# ## 1) training set
plot(rocTest_1$FP, rocTest_1$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[1], 
  xlab="False positive rate", ylab="True positive rate",
  main=paste("Test 1 Year"),
  lwd = 2, cex.main=1.3, cex.lab=1.2, cex.axis=1.2, font=1.2)
aucText=c(aucText,paste0("n=126 Union(20)"," AUC=",sprintf("%.3f",rocTest_1$AUC),""))
abline(0,1)


## 2) Tumor 
aucText=c(aucText,paste0("n=126 Tumor(5)"," AUC=",sprintf("%.3f",rocTest_1_tumor$AUC),""))
lines(rocTest_1_tumor$FP, rocTest_1_tumor$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[2],lwd = 2)


## 3) Immune 
aucText=c(aucText,paste0("n=126 Immune(4)"," AUC=",sprintf("%.3f",rocTest_1_immune$AUC),""))
lines(rocTest_1_immune$FP, rocTest_1_immune$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[3],lwd = 2)

## 4) Cyto
aucText=c(aucText,paste0("n=129 Cytogenetic Risk(3) 1 year"," AUC=",sprintf("%.3f",rocTest_1_Cyto$AUC),""))
lines(rocTest_1_Cyto$FP, rocTest_1_Cyto$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[4],lwd = 2)

## 5) IPSS 
aucText=c(aucText,paste0("n=50 IPSS(3) 1 year"," AUC=",sprintf("%.3f",rocTest_1_IPSS$AUC),""))
lines(rocTest_1_IPSS$FP, rocTest_1_IPSS$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[5],lwd = 2)

# 6) WHO 
aucText=c(aucText,paste0("n=93 WHO(3) 1 year"," AUC=",sprintf("%.3f",rocTest_1_WHO$AUC),""))
lines(rocTest_1_WHO$FP, rocTest_1_WHO$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[6],lwd = 2)


## 7) ELN
aucText=c(aucText,paste0("n=34 ENL2017(3) 1 year"," AUC=",sprintf("%.3f",rocTest_1_ELN$AUC),""))
lines(rocTest_1_ELN$FP, rocTest_1_ELN$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[7],lwd = 2)

legend("bottomright", aucText,lwd=1,bty="n",col=rocTestCol,cex=0.5)
dev.off()

png 
  2

## test-3year

In [41]:
## 1）training set
rocTestCol=rainbow(8)
aucText=c()

output_f = file.path(output_dir,"20250720_4_1_Risk_Model_Build_1307_sample_Test_294_multi_rocTestTest_Test_result_Union20_3_Year_survail.pdf")
pdf(file=output_f,width=6,height=6)
par(oma=c(0.5,1,0,1),font.lab=1.5,font.axis=1.5)

# ## 1) training set
plot(rocTest_3$FP, rocTest_3$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[1], 
  xlab="False positive rate", ylab="True positive rate",
  main=paste("Test 3 year"),
  lwd = 2, cex.main=1.3, cex.lab=1.2, cex.axis=1.2, font=1.2)
aucText=c(aucText,paste0("n=126 Union(20)"," AUC=",sprintf("%.3f",rocTest_3$AUC),""))
abline(0,1)


## 2) Tumor 
aucText=c(aucText,paste0("n=126 Tumor(5)"," AUC=",sprintf("%.3f",rocTest_3_tumor$AUC),""))
lines(rocTest_3_tumor$FP, rocTest_3_tumor$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[2],lwd = 2)


## 3) Immune 
aucText=c(aucText,paste0("n=126 Immune(4)"," AUC=",sprintf("%.3f",rocTest_3_immune$AUC),""))
lines(rocTest_3_immune$FP, rocTest_3_immune$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[3],lwd = 2)

## 4) Cyto
aucText=c(aucText,paste0("n=129 Cytogenetic Risk(3) 3 year"," AUC=",sprintf("%.3f",rocTest_3_Cyto$AUC),""))
lines(rocTest_3_Cyto$FP, rocTest_3_Cyto$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[4],lwd = 2)

## 5) IPSS 
aucText=c(aucText,paste0("n=50 IPSS(3) 3 year"," AUC=",sprintf("%.3f",rocTest_3_IPSS$AUC),""))
lines(rocTest_3_IPSS$FP, rocTest_3_IPSS$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[5],lwd = 2)

# 6) WHO 
aucText=c(aucText,paste0("n=93 WHO(3) 3 year"," AUC=",sprintf("%.3f",rocTest_3_WHO$AUC),""))
lines(rocTest_3_WHO$FP, rocTest_3_WHO$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[6],lwd = 2)


## 7) ELN
aucText=c(aucText,paste0("n=34 ENL2017(3) 3 year"," AUC=",sprintf("%.3f",rocTest_3_ELN$AUC),""))
lines(rocTest_3_ELN$FP, rocTest_3_ELN$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[7],lwd = 2)

legend("bottomright", aucText,lwd=1,bty="n",col=rocTestCol,cex=0.5)
dev.off()

png 
  2

## test-5year

In [56]:
## 1）training set
rocTestCol=rainbow(8)
aucText=c()

output_f = file.path(output_dir,"20250720_4_1_Risk_Model_Build_1307_sample_Test_294_multi_rocTestTest_Test_result_Union20_5_Year_survail.pdf")
pdf(file=output_f,width=6,height=6)
par(oma=c(0.5,1,0,1),font.lab=1.5,font.axis=1.5)

# ## 1) training set
plot(rocTest_5$FP, rocTest_5$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[1], 
  xlab="False positive rate", ylab="True positive rate",
  main=paste("Test 5 year"),
  lwd = 2, cex.main=1.3, cex.lab=1.2, cex.axis=1.2, font=1.2)
aucText=c(aucText,paste0("n=126 Union(20)"," AUC=",sprintf("%.3f",rocTest_5$AUC),""))
abline(0,1)


## 2) Tumor 
aucText=c(aucText,paste0("n=126 Tumor(5)"," AUC=",sprintf("%.3f",rocTest_5_tumor$AUC),""))
lines(rocTest_5_tumor$FP, rocTest_5_tumor$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[2],lwd = 2)


## 3) Immune 
aucText=c(aucText,paste0("n=126 Immune(4)"," AUC=",sprintf("%.3f",rocTest_5_immune$AUC),""))
lines(rocTest_5_immune$FP, rocTest_5_immune$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[3],lwd = 2)

## 4) Cyto
aucText=c(aucText,paste0("n=129 Cytogenetic Risk(3) 5 year"," AUC=",sprintf("%.3f",rocTest_5_Cyto$AUC),""))
lines(rocTest_5_Cyto$FP, rocTest_5_Cyto$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[4],lwd = 2)

## 5) IPSS 
aucText=c(aucText,paste0("n=50 IPSS(3) 5 year"," AUC=",sprintf("%.3f",rocTest_5_IPSS$AUC),""))
lines(rocTest_5_IPSS$FP, rocTest_5_IPSS$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[5],lwd = 2)

# 6) WHO 
aucText=c(aucText,paste0("n=93 WHO(3) 5 year"," AUC=",sprintf("%.3f",rocTest_5_WHO$AUC),""))
lines(rocTest_5_WHO$FP, rocTest_5_WHO$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[6],lwd = 2)


## 7) ELN
aucText=c(aucText,paste0("n=34 ENL2017(3) 5 year"," AUC=",sprintf("%.3f",rocTest_5_ELN$AUC),""))
lines(rocTest_5_ELN$FP, rocTest_5_ELN$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[7],lwd = 2)

legend("bottomright", aucText,lwd=1,bty="n",col=rocTestCol,cex=0.5)
dev.off()

png 
  2

## test-05year

In [8]:
## 1）training set
rocTestCol=rainbow(8)
aucText=c()

output_f = file.path(output_dir,"20250720_4_2_Risk_Model_Build_1307_sample_Test_294_multi_rocTestTest_Test_result_Union20_0.5_Year_survail.pdf")
pdf(file=output_f,width=6,height=6)
par(oma=c(0.5,1,0,1),font.lab=1.5,font.axis=1.5)

# ## 1) training set
plot(rocTest_05$FP, rocTest_05$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[1], 
  xlab="False positive rate", ylab="True positive rate",
  main=paste("Test 0.5 year"),
  lwd = 2, cex.main=1.3, cex.lab=1.2, cex.axis=1.2, font=1.2)
aucText=c(aucText,paste0("n=126 Union(20)"," AUC=",sprintf("%.3f",rocTest_05$AUC),""))
abline(0,1)


## 2) Tumor 
aucText=c(aucText,paste0("n=126 Tumor(5)"," AUC=",sprintf("%.3f",rocTest_05_tumor$AUC),""))
lines(rocTest_05_tumor$FP, rocTest_05_tumor$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[2],lwd = 2)


## 3) Immune 
aucText=c(aucText,paste0("n=126 Immune(4)"," AUC=",sprintf("%.3f",rocTest_05_immune$AUC),""))
lines(rocTest_05_immune$FP, rocTest_05_immune$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[3],lwd = 2)

## 4) Cyto
aucText=c(aucText,paste0("n=129 Cytogenetic Risk(3) 5 year"," AUC=",sprintf("%.3f",rocTest_05_Cyto$AUC),""))
lines(rocTest_05_Cyto$FP, rocTest_05_Cyto$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[4],lwd = 2)

## 5) IPSS 
rocTest_05_IPSS$TP[rocTest_05_IPSS$TP > 1] <- 0.99
aucText=c(aucText,paste0("n=50 IPSS(3) 5 year"," AUC=",sprintf("%.3f",rocTest_05_IPSS$AUC),""))
lines(rocTest_05_IPSS$FP, rocTest_05_IPSS$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[5],lwd = 2)

# 6) WHO 
aucText=c(aucText,paste0("n=93 WHO(3) 5 year"," AUC=",sprintf("%.3f",rocTest_05_WHO$AUC),""))
lines(rocTest_05_WHO$FP, rocTest_05_WHO$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[6],lwd = 2)

## 7) ELN
aucText=c(aucText,paste0("n=34 ENL2017(3) 5 year"," AUC=",sprintf("%.3f",rocTest_05_ELN$AUC),""))
lines(rocTest_05_ELN$FP, rocTest_05_ELN$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[7],lwd = 2)

legend("bottomright", aucText,lwd=1,bty="n",col=rocTestCol,cex=0.5)
dev.off()

png 
  2

## test-2year

In [61]:
## 1）training set
rocTestCol=rainbow(8)
aucText=c()

output_f = file.path(output_dir,"20250720_4_1_Risk_Model_Build_1307_sample_Test_294_multi_rocTestTest_Test_result_Union20_2_Year_survail.pdf")
pdf(file=output_f,width=6,height=6)
par(oma=c(0.5,1,0,1),font.lab=1.5,font.axis=1.5)

# ## 1) training set
plot(rocTest_2$FP, rocTest_2$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[1], 
  xlab="False positive rate", ylab="True positive rate",
  main=paste("Test 2 year"),
  lwd = 2, cex.main=1.3, cex.lab=1.2, cex.axis=1.2, font=1.2)
aucText=c(aucText,paste0("n=126 Union(20)"," AUC=",sprintf("%.3f",rocTest_2$AUC),""))
abline(0,1)


## 2) Tumor 
aucText=c(aucText,paste0("n=126 Tumor(5)"," AUC=",sprintf("%.3f",rocTest_2_tumor$AUC),""))
lines(rocTest_2_tumor$FP, rocTest_2_tumor$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[2],lwd = 2)


## 3) Immune 
aucText=c(aucText,paste0("n=126 Immune(4)"," AUC=",sprintf("%.3f",rocTest_2_immune$AUC),""))
lines(rocTest_2_immune$FP, rocTest_2_immune$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[3],lwd = 2)

## 4) Cyto
aucText=c(aucText,paste0("n=129 Cytogenetic Risk(3) 5 year"," AUC=",sprintf("%.3f",rocTest_2_Cyto$AUC),""))
lines(rocTest_2_Cyto$FP, rocTest_2_Cyto$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[4],lwd = 2)

## 5) IPSS 
aucText=c(aucText,paste0("n=50 IPSS(3) 5 year"," AUC=",sprintf("%.3f",rocTest_2_IPSS$AUC),""))
lines(rocTest_2_IPSS$FP, rocTest_2_IPSS$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[5],lwd = 2)

# 6) WHO 
aucText=c(aucText,paste0("n=93 WHO(3) 5 year"," AUC=",sprintf("%.3f",rocTest_2_WHO$AUC),""))
lines(rocTest_2_WHO$FP, rocTest_2_WHO$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[6],lwd = 2)

## 7) ELN
aucText=c(aucText,paste0("n=34 ENL2017(3) 5 year"," AUC=",sprintf("%.3f",rocTest_2_ELN$AUC),""))
lines(rocTest_2_ELN$FP, rocTest_2_ELN$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[7],lwd = 2)

legend("bottomright", aucText,lwd=1,bty="n",col=rocTestCol,cex=0.5)
dev.off()

png 
  2

## test-4year

In [62]:
## 1）training set
rocTestCol=rainbow(8)
aucText=c()

output_f = file.path(output_dir,"20250720_4_1_Risk_Model_Build_1307_sample_Test_294_multi_rocTestTest_Test_result_Union20_4_Year_survail.pdf")
pdf(file=output_f,width=6,height=6)
par(oma=c(0.5,1,0,1),font.lab=1.5,font.axis=1.5)

# ## 1) training set
plot(rocTest_4$FP, rocTest_4$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[1], 
  xlab="False positive rate", ylab="True positive rate",
  main=paste("Test 4 year"),
  lwd = 2, cex.main=1.3, cex.lab=1.2, cex.axis=1.2, font=1.2)
aucText=c(aucText,paste0("n=126 Union(20)"," AUC=",sprintf("%.3f",rocTest_4$AUC),""))
abline(0,1)


## 2) Tumor 
aucText=c(aucText,paste0("n=126 Tumor(5)"," AUC=",sprintf("%.3f",rocTest_4_tumor$AUC),""))
lines(rocTest_4_tumor$FP, rocTest_4_tumor$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[2],lwd = 2)


## 3) Immune 
aucText=c(aucText,paste0("n=126 Immune(4)"," AUC=",sprintf("%.3f",rocTest_4_immune$AUC),""))
lines(rocTest_4_immune$FP, rocTest_4_immune$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[3],lwd = 2)

## 4) Cyto
aucText=c(aucText,paste0("n=129 Cytogenetic Risk(3) 5 year"," AUC=",sprintf("%.3f",rocTest_4_Cyto$AUC),""))
lines(rocTest_4_Cyto$FP, rocTest_4_Cyto$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[4],lwd = 2)

## 5) IPSS 
aucText=c(aucText,paste0("n=50 IPSS(3) 5 year"," AUC=",sprintf("%.3f",rocTest_4_IPSS$AUC),""))
lines(rocTest_4_IPSS$FP, rocTest_4_IPSS$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[5],lwd = 2)

# 6) WHO 
aucText=c(aucText,paste0("n=93 WHO(3) 5 year"," AUC=",sprintf("%.3f",rocTest_4_WHO$AUC),""))
lines(rocTest_4_WHO$FP, rocTest_4_WHO$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[6],lwd = 2)


## 7) ELN
aucText=c(aucText,paste0("n=34 ENL2017(3) 5 year"," AUC=",sprintf("%.3f",rocTest_4_ELN$AUC),""))
lines(rocTest_4_ELN$FP, rocTest_4_ELN$TP, type="l", xlim=c(0,1), ylim=c(0,1),col=rocTestCol[7],lwd = 2)

legend("bottomright", aucText,lwd=1,bty="n",col=rocTestCol,cex=0.5)
dev.off()

png 
  2

## Save space

In [5]:
if(TRUE){

# output_f = "/share/home/wyz/Project/result/2025/202507/20250718_1_Risk_Model_Build/20250809_1_Risk_Model_Build_Union_420_split_clinlic_ROC/20250809_1_Risk_Model_Build_Union_420_all_workspace.Rdata"

# input_dir = "/share/home/wyz/Project/result/2025/202507/20250718_1_Risk_Model_Build/20250828_1_Risk_Model_Build_Union_420_split_clinlic_ROC"
# output_f = file.path(input_dir,"20250809_1_Risk_Model_Build_Union_420_all_workspace.Rdata")
# load(output_f)
}


In [63]:
if(TRUE){
output_f = file.path(output_dir,"20250828_1_Risk_Model_Build_Union_420_all_add_year_workspace.Rdata")
save.image(file=output_f)

# load(output_f)
}

In [6]:
if(TRUE){
# output_f = "/share/home/wyz/Project/result/2025/202507/20250718_1_Risk_Model_Build/20250828_1_Risk_Model_Build_Union_420_split_clinlic_ROC/20250828_1_Risk_Model_Build_Union_420_all_add_year_workspace.Rdata"
# load(output_f)
}

## Grouping of the line graph

In [10]:
library(survival)
library(survminer)

In [14]:
head(trainRiskOut)

,id,Survival_days,Status_Cox,tumor_to_immune_mark,riskScore,risk
,<chr>,<dbl>,<int>,<fct>,<dbl>,<fct>
AML_Beat_12_00051,AML_Beat_12_00051,1.936986,1,Primitive_CD8T_Int_CD4T_Low,1.6465850,low
AML_Beat_12_00211,AML_Beat_12_00211,3.824658,0,Mature_CD8T_Int_CD4T_Low,2.2629113,high
AML_Beat_13_00157,AML_Beat_13_00157,3.778082,0,Primitive_CD8T_Int_CD4T_Low,1.6465850,low
AML_Beat_13_00163,AML_Beat_13_00163,1.095890,0,GMP_CD8_Int_CD4_High,0.9209025,low
AML_Beat_13_00232,AML_Beat_13_00232,2.742466,0,GMP_CD8T_Int_CD4T_Low,1.3007267,low
AML_Beat_13_00245,AML_Beat_13_00245,2.347945,0,intermediate_CD8T_Int_CD4T_Low,3.5596509,high


In [15]:
trainRiskOut_value_df = trainRiskOut

In [16]:
trainRiskOut_value_df$tumor_to_immune_mark = factor(trainRiskOut_value_df$tumor_to_immune_mark, levels = c(
    "GMP_CD8T_High", 
    "Mature_CD8T_High", 
    "GMP_CD8_Int_CD4_High", 
    "Prog_CD8_Int_CD4_High", 
    "Prog_CD8T_High", 
    "GMP_CD8T_Int_CD4T_Low", 
    "Primitive_CD8T_Int_CD4T_Low", 
    "Prog_CD8T_Low", 
    "Prog_CD8T_Int_CD4T_Low", 
    "Mature_CD8T_Int_CD4T_Low", 
    "intermediate_CD8_Int_CD4_High", 
    "intermediate_CD8T_Int_CD4T_Low", 
    "GMP_CD8T_Low", 
    "Mature_CD8_Int_CD4_High", 
    "Primitive_CD8_Int_CD4_High", 
    "Mature_CD8T_Low", 
    "Primitive_CD8T_High", 
    "intermediate_CD8T_Low", 
    "Primitive_CD8T_Low"
    ))

In [18]:
trainRiskOut_value_df$tumor_to_immune_mark_value = as.numeric(trainRiskOut_value_df$tumor_to_immune_mark)

In [19]:
table(trainRiskOut_value_df$tumor_to_immune_mark_value)


 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 
11 24  9 70  4 26 21  2  3 23  3 12 13  4  6 30  1 22 10 

In [20]:
res.cut <- surv_cutpoint(trainRiskOut_value_df, #
                         time = "Survival_days", #
                         event = "Status_Cox", #
                         variables = c("tumor_to_immune_mark_value") #
                         )

In [21]:
summary(res.cut) 

,cutpoint,statistic
,<dbl>,<dbl>
tumor_to_immune_mark_value,10,8.148015


In [22]:
res.cut 

                           cutpoint statistic
tumor_to_immune_mark_value       10  8.148015

In [23]:
# Draw Forest Diagram
options(forestplot_new_page = FALSE)
output_f = file.path(output_dir,"20250723_10_1_Risk_Model_Build_Union_420.pdf")

pdf(file=output_f,width = 12,height = 12)
plot(res.cut, "tumor_to_immune_mark_value", palette = "npg")
dev.off()

$tumor_to_immune_mark_value


png 
  2

In [ ]:
## Classify the data based on the cutoff points

In [24]:
# 3. Categorize variables
res.cat <- surv_categorize(res.cut)
head(res.cat)

,Survival_days,Status_Cox,tumor_to_immune_mark_value
,<dbl>,<int>,<chr>
AML_Beat_12_00051,1.936986,1,low
AML_Beat_12_00211,3.824658,0,low
AML_Beat_13_00157,3.778082,0,low
AML_Beat_13_00163,1.095890,0,low
AML_Beat_13_00232,2.742466,0,low
AML_Beat_13_00245,2.347945,0,high


In [25]:
class(res.cat)

[1] "data.frame"      "surv_categorize"

In [27]:
res.cat$tumor_to_immune_mark_value = factor(res.cat$tumor_to_immune_mark_value, levels =  c("low","high"))

In [28]:
head(res.cat)

,Survival_days,Status_Cox,tumor_to_immune_mark_value
,<dbl>,<int>,<fct>
AML_Beat_12_00051,1.936986,1,low
AML_Beat_12_00211,3.824658,0,low
AML_Beat_13_00157,3.778082,0,low
AML_Beat_13_00163,1.095890,0,low
AML_Beat_13_00232,2.742466,0,low
AML_Beat_13_00245,2.347945,0,high


In [ ]:
## 

In [26]:
output_f = file.path(output_dir,"20250723_10_1_Risk_Model_Build_Union_420_res.cat.xls")
write.table(res.cat,file=output_f,sep="\t",row.names=F,quote=F)

In [ ]:
## Draw the survival curve

In [30]:
if(TRUE){  
  fit_Group <- survfit(Surv(Survival_days,Status_Cox) ~ risk, data = res.cat) 
  
  #summary(fit_Group)
  
  surv_object = ggsurvplot(fit_Group, data = trainRiskOut,  
                           surv.median.line = "hv",
                           conf.int = FALSE,
                           conf.int.alpha = 0.1,
                           risk.table = TRUE,
                           align = "hv",
                           pval = TRUE,
                           palette = c("#d52331","#2671b2"), 
                           title = paste0("Union(20) Model Risk Group train(n=294)"),  
                           xlab = "Follow up time(d)",
                           legend = "right", # 
                           legend.title = "Group", 
                           break.x.by = 1,  # 
                           risk.table.y.text.col = TRUE, # 
                           risk.table.x.text.angle = 0,  # 
                           risk.table.height = 0.2,     # 
                           font.main = c(8, "bold", "black"),  
  )
  
  library(patchwork)
  plot_object = surv_object$plot / surv_object$table +
    plot_layout(heights = c(5, 1))  
  
  output_f_path = file.path(output_dir,"20250723_10_3_Risk_Model_Build_Union_420_res.cat.pdf")
  ggsave(filename=output_f_path,plot=plot_object,device=pdf,width=180,height=190,units="mm") 
  
}